In [2]:
import numpy as np
import pandas as pd
from netCDF4 import Dataset
import geopandas as gpd
import pyogrio
import xarray as xr
import glob
import os

import py7zr

### Import shared files

In [3]:
#Flowlines (by COMID)
nhd_dir = r'../DATA/NHDPlusPN/NHDPlus17/NHDSnapshot/'
dbfhydro = glob.glob(nhd_dir + '*/*.dbf') 
flowlines_all = gpd.read_file(dbfhydro[0])
flowlines_all.set_index('COMID', inplace = True)
flowlines_all.head(5)

/opt/miniconda3/envs/temp-data-env/lib/python3.12/site-packages/pyogrio/raw.py:198: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D LineString' is converted to 'LineString Z'
  return ogr_read(


,FDATE,RESOLUTION,GNIS_ID,GNIS_NAME,LENGTHKM,REACHCODE,FLOWDIR,WBAREACOMI,FTYPE,FCODE,SHAPE_LENG,ENABLED,GNIS_NBR,geometry
COMID,,,,,,,,,,,,,,
9301535,1999-10-08,Medium,None,None,0.999,09040002011583,Uninitialized,0,StreamRiver,46006,0.011661,True,0,"LINESTRING Z (-114.05855 48.99999 0, -114.0583..."
22968128,2001-03-02,Medium,None,None,0.134,10030104009812,Uninitialized,0,StreamRiver,46003,0.001671,True,0,"LINESTRING Z (-113.00089 47.46274 0, -113.0020..."
22968138,2001-03-02,Medium,None,None,0.217,10030104009813,Uninitialized,0,StreamRiver,46003,0.002145,True,0,"LINESTRING Z (-113.0013 47.45837 0, -113.00137..."
23135995,2001-02-07,Medium,None,None,0.916,16010102003533,Uninitialized,0,StreamRiver,46003,0.010671,True,0,"LINESTRING Z (-111.06214 42.50841 0, -111.0619..."
23135997,2001-02-07,Medium,None,None,1.865,16010102003534,Uninitialized,0,StreamRiver,46003,0.019193,True,0,"LINESTRING Z (-111.04516 42.51372 0, -111.0447..."


In [4]:
# Spatial metrics (time invariant)
spatial_data_all = pd.read_csv('../DATA/Stream_Temp/spatial_data.csv')
spatial_data_fill = spatial_data_all.fillna(-9999)
spatial_data_fill.head(5)

,COMID,cov.length_km,cov.stream_order,cov.path_length,cov.tot.da.sqkm,cov.elev_max_smo,cov.elev_min_smo,cov.slope,cov.sinuosity,cov.lat_v,...,cov.pct_urb_all_ws,cov.canopy_rca,cov.canopy_line,cov.area_sqkm,cov.canopy_ws,cov.dam_name,cov.proportion_dam_influenced,cov.distance_below_dam,cov.dam_hgt_m,cov.dam_main_use
0,9301535,0.999,-9999.0,-9999.000,-9999.0000,-9999.0,-9999.0,-9999.000000,1.47,-9999.000000,...,-9999.000000,-9999.0,-9999.0,-9999.0000,-9999.000000,-9999,-9999.000000,-9999.000,-9999.0,-9999
1,22226592,0.733,-9999.0,-9999.000,-9999.0000,-9999.0,-9999.0,-9999.000000,1.13,-9999.000000,...,-9999.000000,-9999.0,-9999.0,-9999.0000,-9999.000000,-9999,-9999.000000,-9999.000,-9999.0,-9999
2,22226600,6.117,-9999.0,-9999.000,-9999.0000,-9999.0,-9999.0,-9999.000000,1.62,-9999.000000,...,-9999.000000,-9999.0,-9999.0,-9999.0000,-9999.000000,-9999,-9999.000000,-9999.000,-9999.0,-9999
3,22877591,0.247,6.0,1290.195,28030.5369,56102.0,55932.0,0.006883,1.00,48.558399,...,0.946698,89.0,90.0,1.9161,88.592748,Libby,0.831332,71.294,129.0,Hydroelectricity
4,22877593,0.055,3.0,1290.442,58.4136,56129.0,56102.0,0.004909,1.00,48.556999,...,0.000000,90.0,90.0,0.4284,90.227047,-9999,-9999.000000,-9999.000,-9999.0,-9999


In [5]:
covariate_metadata = pd.read_csv('../DATA/Stream_Temp/covariate_metadata.csv', nrows = 80)
covariate_metadata.head(5)

,COVARIATE,UNITS,SYMBOL,DESCRIPTION,DATA SOURCE (Retrospective),DATA SOURCE (GCM Scenarios)
0,COMID,unitless,NaN,unique identifier for matching to stream reach...,NHDv2 attributes; McKay et al. (2012),NaN
1,cov.antec_air_temp,C,Tl,Antecedent air temperature (summarized by the ...,Calculated; Siegel et al. (2023),Calculated as in Siegel et al. (2023)
2,cov.air_temp_mean_ws,C,Tws,mean daily air temperature in upstream watershed,PRISM; Di Luzio et al. (2008),RMJOC(2018); IPCC (2014); USCRT
3,cov.SWE_ws,mm,Sws,Mean daily snowpack snow-water equivalent (SWE...,National Snow and Ice Data Center; Broxton et ...,RMJOC(2018) intermediate modeling product
4,cov.daylength_hours,h,DL,daylength,DayMet; https://daymet.ornl.gov/,DayMet; https://daymet.ornl.gov/


In [38]:
[str(i) for i in (set(data.columns)).difference(set(covariate_metadata.COVARIATE))]

['cov.air_temp_ws',
 'cov.NWM_flow_log',
 'cov.air_temp_3d',
 'cov.NWM_flow',
 'cov.air_temp_18d',
 'tim.date',
 'cov.air_temp_24d',
 'cov.SWE_mean_year',
 'cov.air_temp_27d',
 'cov.air_temp_15d',
 'tim.year',
 'cov.daylength',
 'lookup',
 'cov.air_temp_9d',
 'cov.air_temp',
 'cov.air_temp_21d',
 'cov.air_temp_35d',
 'cov.air_temp_6d',
 'cov.SWE',
 'cov.air_temp_12d',
 'Huc10',
 'cov.air_temp_50d',
 'cov.air_temp_40d',
 'cov.air_temp_60d',
 'cov.air_temp_30d',
 'cov.SWE_last_doy']

In [39]:
len(data.columns)


31

In [6]:
covariate_metadata[covariate_metadata.COVARIATE.str.contains('SWE')==True]

,COVARIATE,UNITS,SYMBOL,DESCRIPTION,DATA SOURCE (Retrospective),DATA SOURCE (GCM Scenarios)
3,cov.SWE_ws,mm,Sws,Mean daily snowpack snow-water equivalent (SWE...,National Snow and Ice Data Center; Broxton et ...,RMJOC(2018) intermediate modeling product
7,cov.SWE_1Apr,mm,SA1,April 1st watershed snow water equivalent,Calculated; Siegel et al. (2023),Calculated as in Siegel et al. (2023)


In [8]:
gcm_metadata = pd.read_csv('../DATA/Stream_Temp/gcm_metadata.csv', nrows = 10)
gcm_metadata.head(5)

,Global Climate Model short name,GCM long name,Country,Agency,Coupled Model Intercomparison Project version,Emissions scenario,Downscaling method,Hydrology model,Network routing
0,CanESM2,Canadian Earth System Model version 2,Canada,Canadian Centre for Climate Modeling and Analysis,5,RCP 8.5,Multivariate Adaptive Constructed Analogs (Aba...,"Variable Infiltration Capacity, using paramete...",mizuRoute (Mizukami 2019)
1,CCSM4,Community Climate System Model version 4,USA,"National Center of Atmospheric Research, USA",5,RCP 8.5,Multivariate Adaptive Constructed Analogs (Aba...,"Variable Infiltration Capacity, using paramete...",mizuRoute (Mizukami 2019)
2,CNRM-CM5,National Center for Meteorological Research Mo...,France,"National Centre of Meteorological Research, Fr...",5,RCP 8.5,Multivariate Adaptive Constructed Analogs (Aba...,"Variable Infiltration Capacity, using paramete...",mizuRoute (Mizukami 2019)
3,CSIRO-Mk3-6-0,Commonwealth Scientific and Industrial Researc...,Australia,Commonwealth Scientific and Industrial Researc...,5,RCP 8.5,Multivariate Adaptive Constructed Analogs (Aba...,"Variable Infiltration Capacity, using paramete...",mizuRoute (Mizukami 2019)
4,GDFL-ESM2M,Global Fluid Dynamics Lab's Earth Systems Mode...,USA,"NOAA Geophysical Fluid Dynamics Laboratory, USA",5,RCP 8.5,Multivariate Adaptive Constructed Analogs (Aba...,"Variable Infiltration Capacity, using paramete...",mizuRoute (Mizukami 2019)


In [9]:
gcms = list(gcm_metadata.iloc[:, 0].values)

shortname_dict = {i+1:gcms[i] for i in list(range(10))}
shortname_dict

{1: 'CanESM2',
 2: 'CCSM4',
 3: 'CNRM-CM5',
 4: 'CSIRO-Mk3-6-0',
 5: 'GDFL-ESM2M',
 6: 'HadGEM-CC365',
 7: 'HadGEM2-ES',
 8: 'inmcm4',
 9: 'IPSL-CM5A-MR',
 10: 'MIROC5'}

In [10]:
# the following use ~ mm/21/yy dates for cutting (i.e. spring starts on March 21) - per Aimee

def assign_season(doy):
    """Assigns a season based on the day of year (doy)."""
    if 1 <= doy <= 79:
        return "winter"
    elif 80 <= doy <= 171:
        return "spring"
    elif 172 <= doy <= 263:
        return "summer"
    elif 264 <= doy <= 356:
        return "fall"
    else:
        return "winter"

In [11]:
temp_dir = '/Volumes/Elements/URycki/temp_data/'

In [12]:
periods = {'2020s': list(range(2020, 2030)), 
           '1990s': list(range(1990, 2000)), 
           '2050s': list(range(2050, 2060)), 
           '2080s': list(range(2080, 2090))}

In [13]:
temp_dir_gcms = os.path.join(temp_dir, 'preds_GCM/')
temp_files_gcms = glob.glob(temp_dir_gcms + '*.nc')
print(len(temp_files_gcms))
temp_files_gcms.sort()

1316


In [14]:
def getSeasonalAnomolies(indf, period_grps: dict):
    cur_huc_anoms = []
    cur_comids = list(indf.COMID.unique())
    for j in cur_comids:
        cur_preds = indf[indf.COMID == j]
        cur_ens_med = indf.groupby('date').median().drop('GCM', axis =1)
        cur_ens_med['doy'] = cur_ens_med.index.dayofyear
        cur_ens_med['season'] = cur_ens_med['doy'].apply(assign_season)

        cur_anomolies = {}
        p = 0
        for per in list(period_grps.keys()):
            if p == 0: cur_anomolies[per] = cur_ens_med[cur_ens_med.index.year.isin(periods[per])].groupby('season').median()['prd.stream_temp']
            else: cur_anomolies[per] = cur_ens_med[cur_ens_med.index.year.isin(periods[per])].groupby('season').median()['prd.stream_temp'].subtract(cur_anomolies['2020s'])
            p +=1
        cur_seas_anomolies = pd.DataFrame(cur_anomolies)
        cur_huc_anoms.append(cur_seas_anomolies)
    return pd.concat(cur_huc_anoms, keys = cur_comids)

In [16]:
# Temp predictions
for i in range(len(temp_files_gcms)):
    cur_file = temp_files_gcms[i]
    cur_huc = int(cur_file.split('/')[-1][:-3])

    preds = xr.open_dataset(cur_file, decode_times=True)
    predsdf = preds.to_dataframe(dim_order = ['COMID', 'GCM', 'date'])
    preds_flat = predsdf.reset_index()

    if predsdf.shape[0] > 0:
        cur_huc_anoms = []
        cur_comids = list(preds_flat.COMID.unique())
        for j in cur_comids:
            cur_preds = preds_flat[preds_flat.COMID == j]
            cur_ens_med = cur_preds.groupby('date').median().drop('GCM', axis =1)
            cur_ens_med['doy'] = cur_ens_med.index.dayofyear
            cur_ens_med['season'] = cur_ens_med['doy'].apply(assign_season)
    
            cur_anomolies = {}
            for dec in list(periods.keys()):
                if dec == '2020s': cur_anomolies[dec] = cur_ens_med[cur_ens_med.index.year.isin(periods[dec])].groupby('season').median()['prd.stream_temp']
                else: cur_anomolies[dec] = cur_ens_med[cur_ens_med.index.year.isin(periods[dec])].groupby('season').median()['prd.stream_temp'].subtract(cur_anomolies['2020s'])
    
            cur_seas_anomolies = pd.DataFrame(cur_anomolies)
            cur_huc_anoms.append(cur_seas_anomolies)
        
        huc_anoms = pd.concat(cur_huc_anoms, keys = cur_comids)
        huc_anoms = huc_anoms.reset_index(names = ['COMID', 'season'])
        huc_anoms.to_csv(os.path.join(temp_dir_gcms, 'seasonal_anomolies', f'{cur_huc}_anoms.csv'))
        with py7zr.SevenZipFile(os.path.join(temp_dir_gcms, 'seasonal_anomolies', f'{cur_huc}_anoms.7z'), 'w') as archive:
            archive.write(os.path.join(temp_dir_gcms, 'seasonal_anomolies', f'{cur_huc}_anoms.csv'))
        os.remove(os.path.join(temp_dir_gcms, 'seasonal_anomolies', f'{cur_huc}_anoms.csv')) 

In [15]:
temp_files_anoms = glob.glob('/Volumes/Elements/URycki/temp_data/preds_GCM/seasonal_anomolies/*.7z')

In [16]:
hucs = [f[-19:-9] for f in temp_files_anoms]
#hucs

In [17]:
cur_huc = hucs[0]
cur_huc

'1701010107'

In [17]:
len(temp_files_gcms) == len(temp_files_anoms)

False

In [18]:
test = temp_files_gcms[0]
test.split('/')[-1][:-3]

'1701010107'

In [19]:
nc_list = [h.split('/')[-1][:-3] for h in temp_files_gcms]

In [20]:
temp_anom_list = [a.split('/')[-1][:-9] for a in temp_files_anoms]
temp_anom_list[0]

'1701010107'

In [20]:
diffs = list(set(nc_list).difference(set(temp_anom_list)))
diffs

['1702001403',
 '1701021310',
 '1711001904',
 '1711001901',
 '1705012305',
 '1707030508']

In [21]:
for diff in diffs:
    preds = xr.open_dataset(f'/Volumes/Elements/URycki/temp_data/preds_GCM/{diff}.nc', decode_times=True)
    predsdf = preds.to_dataframe(dim_order = ['COMID', 'GCM', 'date'])
    preds_flat = predsdf.reset_index()
    print(diff, ': ', len(preds_flat))

1702001403 :  0
1701021310 :  0
1711001904 :  0
1711001901 :  0
1705012305 :  0
1707030508 :  0


### Check result with test case

In [61]:
k =2

In [83]:
cur_file_ck = temp_files_gcms[k]
cur_file_ck

'/Volumes/Elements/URycki/temp_data/preds_GCM/1701010109.nc'

In [84]:
preds_ck = xr.open_dataset(cur_file_ck, decode_times=True)
predsdf_ck = preds_ck.to_dataframe(dim_order = ['COMID', 'GCM', 'date'])
preds_flat_ck = predsdf_ck.reset_index()
preds_flat_ck

,COMID,GCM,date,prd.stream_temp
0,22878617,1,1950-01-01,NaN
1,22878617,1,1950-01-02,NaN
2,22878617,1,1950-01-03,0.000000
3,22878617,1,1950-01-04,0.076190
4,22878617,1,1950-01-05,0.000000
...,...,...,...,...
4382955,22882373,10,2099-12-27,3.492679
4382956,22882373,10,2099-12-28,3.853480
4382957,22882373,10,2099-12-29,3.273735
4382958,22882373,10,2099-12-30,5.010366


In [85]:
cur_com = preds_flat_ck.COMID.unique()[-1]
cur_com

np.int32(22882373)

In [65]:
cur_preds_ck = preds_flat_ck[preds_flat_ck.COMID == cur_com]
cur_preds_ck

,COMID,GCM,date,prd.stream_temp
3835090,22882373,1,1950-01-01,NaN
3835091,22882373,1,1950-01-02,NaN
3835092,22882373,1,1950-01-03,NaN
3835093,22882373,1,1950-01-04,NaN
3835094,22882373,1,1950-01-05,NaN
...,...,...,...,...
4382955,22882373,10,2099-12-27,3.492679
4382956,22882373,10,2099-12-28,3.853480
4382957,22882373,10,2099-12-29,3.273735
4382958,22882373,10,2099-12-30,5.010366


In [66]:
cur_preds_ck[cur_preds_ck.date == '12/31/2099']

,COMID,GCM,date,prd.stream_temp
3889876,22882373,1,2099-12-31,1.660377
3944663,22882373,2,2099-12-31,2.548640
3999450,22882373,3,2099-12-31,1.288398
4054237,22882373,4,2099-12-31,3.819269
4109024,22882373,5,2099-12-31,1.723519
4163811,22882373,6,2099-12-31,5.566496
4218598,22882373,7,2099-12-31,4.049389
4273385,22882373,8,2099-12-31,2.672233
4328172,22882373,9,2099-12-31,NaN
4382959,22882373,10,2099-12-31,4.310524


In [67]:
np.nanmedian(cur_preds_ck[cur_preds_ck.date == '12/31/2099']['prd.stream_temp'])

np.float64(2.67223326861739)

In [68]:
cur_ens_med_ck = cur_preds_ck.groupby('date').median().drop('GCM', axis =1)
cur_ens_med_ck['doy'] = cur_ens_med_ck.index.dayofyear
cur_ens_med_ck['season'] = cur_ens_med_ck['doy'].apply(assign_season)
cur_ens_med_ck

,COMID,prd.stream_temp,doy,season
date,,,,
1950-01-01,22882373.0,NaN,1,winter
1950-01-02,22882373.0,NaN,2,winter
1950-01-03,22882373.0,0.109532,3,winter
1950-01-04,22882373.0,0.132757,4,winter
1950-01-05,22882373.0,0.223510,5,winter
...,...,...,...,...
2099-12-27,22882373.0,2.490783,361,winter
2099-12-28,22882373.0,2.087694,362,winter
2099-12-29,22882373.0,2.058112,363,winter


In [69]:
vals = cur_ens_med_ck[(cur_ens_med_ck.index.year.isin(periods['2020s'])) & (cur_ens_med_ck.season == 'winter')]['prd.stream_temp'].values

In [70]:
np.nanmedian(vals)

np.float64(1.32164705893292)

In [71]:
np.median(vals)

np.float64(1.32164705893292)

In [72]:
cur = cur_ens_med_ck[cur_ens_med_ck.index.year.isin(periods['2020s'])].groupby('season').median()['prd.stream_temp']
cur

season
fall       3.363799
spring     7.035941
summer    13.860170
winter     1.321647
Name: prd.stream_temp, dtype: float64

In [73]:
fifties = cur_ens_med_ck[cur_ens_med_ck.index.year.isin(periods['2050s'])].groupby('season').median()['prd.stream_temp']
fifties

season
fall       4.102839
spring     8.075323
summer    14.787528
winter     1.470753
Name: prd.stream_temp, dtype: float64

In [74]:
nineties = cur_ens_med_ck[cur_ens_med_ck.index.year.isin(periods['1990s'])].groupby('season').median()['prd.stream_temp']
nineties

season
fall       3.158271
spring     6.414733
summer    12.771167
winter     1.143975
Name: prd.stream_temp, dtype: float64

In [75]:
nineties-cur

season
fall     -0.205528
spring   -0.621208
summer   -1.089002
winter   -0.177672
Name: prd.stream_temp, dtype: float64

In [76]:
fifties-cur

season
fall      0.739040
spring    1.039383
summer    0.927359
winter    0.149106
Name: prd.stream_temp, dtype: float64

In [90]:
cur_file_ck

'/Volumes/Elements/URycki/temp_data/preds_GCM/1701010109.nc'

In [91]:
cur_huc = cur_file_ck.split('/')[-1][:-3]
cur_huc

'1701010109'

In [ ]:
with py7zr.SevenZipFile(f'/Volumes/Elements/URycki/temp_data/preds_GCM/seasonal_anomolies/{cur_huc}_anoms.7z', mode='r') as archive:
    archive.extractall(path = '/.')

ck = pd.read_csv(f'/Volumes/Elements/URycki/temp_data/preds_GCM/seasonal_anomolies/{cur_huc}_anoms.csv')

ck.rename(columns={'Unnamed: 0': "COMID"}, inplace = True)

In [133]:
ck[ck.COMID == cur_com]

,COMID,season,2020s,1990s,2050s,2080s
28,22882373,fall,3.363799,-0.205528,0.739040,1.731192
29,22882373,spring,7.035941,-0.621208,1.039383,2.181650
30,22882373,summer,13.860170,-1.089002,0.927359,1.970110
31,22882373,winter,1.321647,-0.177672,0.149106,0.777069


In [148]:
ck[ck.COMID == cur_com].set_index('season')['2020s'].subtract(cur)

season
fall      0.0
spring    0.0
summer    0.0
winter    0.0
dtype: float64

In [146]:
ck[ck.COMID == cur_com].set_index('season')['1990s']

season
fall     -0.205528
spring   -0.621208
summer   -1.089002
winter   -0.177672
Name: 1990s, dtype: float64

In [147]:
ck[ck.COMID == cur_com].set_index('season')['1990s'].subtract(nineties-cur)

season
fall      0.000000e+00
spring    0.000000e+00
summer    0.000000e+00
winter    2.775558e-17
dtype: float64

In [149]:
ck[ck.COMID == cur_com].set_index('season')['2050s'].subtract(fifties-cur)

season
fall      0.000000e+00
spring    0.000000e+00
summer   -1.110223e-16
winter   -5.551115e-17
dtype: float64

### Check done (ok).

In [22]:
nc_metadata = preds.attrs
nc_metadata

{'title': 'Predicted future stream temperatures',
 'institution': 'https://www.fisheries.noaa.gov/region/west-coast/northwest-science',
 'source': 'URL TBD',
 'references': 'Fullerton et al. (2024). NOAA Data Report',
 'history': 'A. H. Fullerton Sun Oct 27 11:52:08 2024',
 'Conventions': 'CF-1.0'}

In [17]:
cur_output = (f'DATA/gpkgs/{huc}.gpkg')

In [18]:
cur_output

'DATA/gpkgs/1702000720.gpkg'

In [24]:
cur_comids_preds = preds_flat.COMID.unique()
len(cur_comids_preds)

42

In [18]:
%load_ext rpy2.ipython

In [19]:
%%R
install.packages("fst")

R[write to console]: Installing package into ‘/Users/dawn.urycki/Library/R/x86_64/4.3/library’
(as ‘lib’ is unspecified)



--- Please select a CRAN mirror for use in this session ---
Secure CRAN mirrors 

 1: 0-Cloud [https]
 2: Australia (Canberra) [https]
 3: Australia (Melbourne 1) [https]
 4: Australia (Melbourne 2) [https]
 5: Austria (Wien 1) [https]
 6: Belgium (Brussels) [https]
 7: Brazil (PR) [https]
 8: Brazil (SP 1) [https]
 9: Brazil (SP 2) [https]
10: Bulgaria [https]
11: Canada (MB) [https]
12: Canada (ON 1) [https]
13: Canada (ON 2) [https]
14: Chile (Santiago) [https]
15: China (Beijing 2) [https]
16: China (Beijing 3) [https]
17: China (Hefei) [https]
18: China (Hong Kong) [https]
19: China (Jinan) [https]
20: China (Lanzhou) [https]
21: China (Nanjing) [https]
22: China (Shanghai 2) [https]
23: China (Shenzhen) [https]
24: China (Wuhan) [https]
25: Colombia (Cali) [https]
26: Costa Rica [https]
27: Cyprus [https]
28: Czech Republic [https]
29: Denmark [https]
30: East Asia [https]
31: Ecuador (Cuenca) [https]
32: France (Lyon 1) [https]
33: France (Lyon 2) [https]
34: France (Marseille) 

Selection:  70


R[write to console]: trying URL 'https://mirrors.nics.utk.edu/cran/src/contrib/fst_0.9.8.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 274495 bytes (268 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to co

x86_64-apple-darwin13.4.0-clang++ -std=gnu++17 -I"/opt/miniconda3/envs/temp-data-env/lib/R/include" -DNDEBUG  -I'/Users/dawn.urycki/Library/R/x86_64/4.3/library/Rcpp/include' -I'/Users/dawn.urycki/Library/R/x86_64/4.3/library/fstcore/include' -D_FORTIFY_SOURCE=2 -isystem /opt/miniconda3/envs/temp-data-env/include -mmacosx-version-min=10.13 -mmacosx-version-min=10.13 -I/opt/miniconda3/envs/temp-data-env/include    -fPIC  -march=core2 -mtune=haswell -mssse3 -ftree-vectorize -fPIC -fstack-protector-strong -O2 -pipe -stdlib=libc++ -fvisibility-inlines-hidden -fmessage-length=0 -isystem /opt/miniconda3/envs/temp-data-env/include -fdebug-prefix-map=/Users/runner/miniforge3/conda-bld/r-base-split_1728291276454/work=/usr/local/src/conda/r-base-4.3.3 -fdebug-prefix-map=/opt/miniconda3/envs/temp-data-env=/usr/local/src/conda-prefix  -c RcppExports.cpp -o RcppExports.o
x86_64-apple-darwin13.4.0-clang++ -std=gnu++17 -I"/opt/miniconda3/envs/temp-data-env/lib/R/include" -DNDEBUG  -I'/Users/dawn.uryc

installing to /Users/dawn.urycki/Library/R/x86_64/4.3/library/00LOCK-fst/00new/fst/libs
** R
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
** building package indices
** testing if installed package can be loaded from temporary location
** checking absolute paths in shared objects and dynamic libraries
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (fst)
R[write to console]: 

R[write to console]: 
R[write to console]: The downloaded source packages are in
	‘/private/var/folders/95/99dy_ns968195cnlvp5xhwqm0000gp/T/RtmplCsQtC/downloaded_packages’
R[write to console]: 
R[write to console]: 



In [20]:
%%R
library(fst)
getwd()

R[write to console]: fst package v0.9.8



[1] "/Users/dawn.urycki/Repos/temp-data"


In [21]:
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
pandas2ri.activate()

In [22]:
[f.name for f in os.scandir('/Volumes/Elements/URycki/temp_data/covariates_GCM/') if f.is_dir()]

['CanESM2',
 'GDFL-ESM2M',
 'Livneh',
 'HadGEM-CC365',
 'CNRM-CM5',
 'IPSL-CM5A-MR',
 'HadGEM2-ES',
 'inmcm4',
 'CCSM4',
 'CSIRO-Mk3-6-0',
 'MIROC5']

In [23]:
gcms

['CanESM2',
 'CCSM4',
 'CNRM-CM5',
 'CSIRO-Mk3-6-0',
 'GDFL-ESM2M',
 'HadGEM-CC365',
 'HadGEM2-ES',
 'inmcm4',
 'IPSL-CM5A-MR',
 'MIROC5']

In [24]:
cur_gcm = gcms[0]
cur_gcm

'CanESM2'

In [25]:
cur_huc

'1701010107'

In [26]:
data = ro.r(f"read.fst('/Volumes/Elements/URycki/temp_data/cov_retro/huc_{cur_huc}.fst')")

R[write to console]: fstcore package v0.9.18

R[write to console]: (OpenMP detected, using 16 threads)



In [28]:
data.columns

Index(['lookup', 'COMID', 'cov.air_temp', 'cov.air_temp_ws', 'cov.air_temp_3d',
       'cov.air_temp_6d', 'cov.air_temp_9d', 'cov.air_temp_12d',
       'cov.air_temp_15d', 'cov.air_temp_18d', 'cov.air_temp_21d',
       'cov.air_temp_24d', 'cov.air_temp_27d', 'cov.air_temp_30d',
       'cov.air_temp_35d', 'cov.air_temp_40d', 'cov.air_temp_50d',
       'cov.air_temp_60d', 'Huc10', 'tim.year', 'tim.doy', 'tim.date',
       'cov.SWE', 'cov.SWE_ws', 'cov.NWM_flow', 'cov.NWM_flow_log',
       'cov.daylength', 'cov.daylength_hours', 'cov.SWE_mean_year',
       'cov.SWE_1Apr', 'cov.SWE_last_doy'],
      dtype='object')

In [32]:
%%time
gcm_frames = []
fail_list = []
for cur_gcm in gcms:
    try: gcm_frames.append(ro.r(f"read.fst('/Volumes/Elements/URycki/temp_data/covariates_GCM/{cur_gcm}/huc_{cur_huc}.fst')"))
    except: fail_list.append(cur_gcm)

    

R[write to console]: fstcore package v0.9.18

R[write to console]: (OpenMP detected, using 16 threads)



CPU times: user 2min 47s, sys: 2min, total: 4min 48s
Wall time: 1min 28s


In [33]:
fail_list

[]

In [34]:
len(gcm_frames)

10

In [40]:
df = pd.concat(gcm_frames, keys = gcms, names = ['GCM'])
df.head()

COMID  SegID     lookup  cov.air_temp  cov.air_temp_ws  \
GCM                                                                    
CanESM2 1  22878641     35  35_1950_1    -23.876358       -23.677581   
        2  22878643     35  35_1950_1    -23.876358       -23.677581   
        3  22878645     35  35_1950_1    -23.876358       -23.677581   
        4  22878647     35  35_1950_1    -23.876358       -23.677581   
        5  22878649     35  35_1950_1    -23.876358       -23.677581   

           cov.air_temp_3d  cov.air_temp_6d  cov.air_temp_9d  \
GCM                                                            
CanESM2 1              NaN              NaN              NaN   
        2              NaN              NaN              NaN   
        3              NaN              NaN              NaN   
        4              NaN              NaN              NaN   
        5              NaN              NaN              NaN   

           cov.air_temp_12d  cov.air_temp_15d  ...  cov.Flow  cov.Flow_log  \
GCM                                            ...                           
CanESM2 1               NaN               NaN  ...  1.650324      0.500972   
        2               NaN               NaN  ...  1.650324      0.500972   
        3               NaN               NaN  ...  1.650324      0.500972   
        4               NaN               NaN  ...  1.650324      0.500972   
        5               NaN               NaN  ...  1.650324      0.500972   

           tim.year  tim.doy  tim.date  cov.daylength  cov.daylength_hours  \
GCM                                                                          
CanESM2 1    1950.0      1.0   -7305.0        29376.0                 8.16   
        2    1950.0      1.0   -7305.0        29376.0                 8.16   
        3    1950.0      1.0   -7305.0        29376.0                 8.16   
        4    1950.0      1.0   -7305.0        29376.0                 8.16   
        5    1950.0      1.0   -7305.0        29376.0                 8.16   

           cov.SWE_mean_year  cov.SWE_1Apr  cov.SWE_last_doy  
GCM                                                           
CanESM2 1          68.982662     177.26699             365.0  
        2          68.982662     177.26699             365.0  
        3          68.982662     177.26699             365.0  
        4          68.982662     177.26699             365.0  
        5          68.982662     177.26699             365.0  

[5 rows x 31 columns]

In [242]:
df.loc[df.COMID==22878621].index.get_level_values(0).unique() #check

Index(['CanESM2', 'CCSM4', 'CNRM-CM5', 'CSIRO-Mk3-6-0', 'GDFL-ESM2M',
       'HadGEM-CC365', 'HadGEM2-ES', 'inmcm4', 'IPSL-CM5A-MR', 'MIROC5'],
      dtype='object', name='GCM')

In [42]:
df['date'] = pd.to_datetime(df['tim.year'] * 1000 + df['tim.doy'], format='%Y%j')

In [43]:
df

COMID  SegID      lookup  cov.air_temp  cov.air_temp_ws  \
GCM                                                                           
CanESM2 1        22878641     35   35_1950_1    -23.876358       -23.677581   
        2        22878643     35   35_1950_1    -23.876358       -23.677581   
        3        22878645     35   35_1950_1    -23.876358       -23.677581   
        4        22878647     35   35_1950_1    -23.876358       -23.677581   
        5        22878649     35   35_1950_1    -23.876358       -23.677581   
...                   ...    ...         ...           ...              ...   
MIROC5  1698393  22878661     36  36_2099_98     10.096290        10.096290   
        1698394  22878663     36  36_2099_98     10.096290        10.096290   
        1698395  22878659     36  36_2099_99     10.749942        10.749942   
        1698396  22878661     36  36_2099_99     10.749942        10.749942   
        1698397  22878663     36  36_2099_99     10.749942        10.749942   

                 cov.air_temp_3d  cov.air_temp_6d  cov.air_temp_9d  \
GCM                                                                  
CanESM2 1                    NaN              NaN              NaN   
        2                    NaN              NaN              NaN   
        3                    NaN              NaN              NaN   
        4                    NaN              NaN              NaN   
        5                    NaN              NaN              NaN   
...                          ...              ...              ...   
MIROC5  1698393         9.225973         8.913307         8.496898   
        1698394         9.225973         8.913307         8.496898   
        1698395        10.188242         8.888762         9.179569   
        1698396        10.188242         8.888762         9.179569   
        1698397        10.188242         8.888762         9.179569   

                 cov.air_temp_12d  cov.air_temp_15d  ...  cov.Flow_log  \
GCM                                                  ...                 
CanESM2 1                     NaN               NaN  ...      0.500972   
        2                     NaN               NaN  ...      0.500972   
        3                     NaN               NaN  ...      0.500972   
        4                     NaN               NaN  ...      0.500972   
        5                     NaN               NaN  ...      0.500972   
...                           ...               ...  ...           ...   
MIROC5  1698393          8.135863          7.891751  ...      2.101081   
        1698394          8.135863          7.891751  ...      2.101081   
        1698395          8.326978          8.111423  ...      2.113511   
        1698396          8.326978          8.111423  ...      2.113511   
        1698397          8.326978          8.111423  ...      2.113511   

                 tim.year  tim.doy  tim.date  cov.daylength  \
GCM                                                           
CanESM2 1          1950.0      1.0   -7305.0        29376.0   
        2          1950.0      1.0   -7305.0        29376.0   
        3          1950.0      1.0   -7305.0        29376.0   
        4          1950.0      1.0   -7305.0        29376.0   
        5          1950.0      1.0   -7305.0        29376.0   
...                   ...      ...       ...            ...   
MIROC5  1698393    2099.0     98.0   47214.0        47001.6   
        1698394    2099.0     98.0   47214.0        47001.6   
        1698395    2099.0     99.0   47215.0        47001.6   
        1698396    2099.0     99.0   47215.0        47001.6   
        1698397    2099.0     99.0   47215.0        47001.6   

                 cov.daylength_hours  cov.SWE_mean_year  cov.SWE_1Apr  \
GCM                                                                     
CanESM2 1                      8.160          68.982662    177.266990   
        2                      8.160          68.982662    177.266990   
      

In [44]:
df.tail(30)

COMID  SegID      lookup  cov.air_temp  cov.air_temp_ws  \
GCM                                                                          
MIROC5 1698368  22878659     36  36_2099_90      4.605903         4.605903   
       1698369  22878661     36  36_2099_90      4.605903         4.605903   
       1698370  22878663     36  36_2099_90      4.605903         4.605903   
       1698371  22878659     36  36_2099_91      8.297876         8.297876   
       1698372  22878661     36  36_2099_91      8.297876         8.297876   
       1698373  22878663     36  36_2099_91      8.297876         8.297876   
       1698374  22878659     36  36_2099_92     10.088462        10.088462   
       1698375  22878661     36  36_2099_92     10.088462        10.088462   
       1698376  22878663     36  36_2099_92     10.088462        10.088462   
       1698377  22878659     36  36_2099_93     10.897214        10.897214   
       1698378  22878661     36  36_2099_93     10.897214        10.897214   
       1698379  22878663     36  36_2099_93     10.897214        10.897214   
       1698380  22878659     36  36_2099_94      8.551267         8.551267   
       1698381  22878661     36  36_2099_94      8.551267         8.551267   
       1698382  22878663     36  36_2099_94      8.551267         8.551267   
       1698383  22878659     36  36_2099_95      6.353443         6.353443   
       1698384  22878661     36  36_2099_95      6.353443         6.353443   
       1698385  22878663     36  36_2099_95      6.353443         6.353443   
       1698386  22878659     36  36_2099_96      7.863135         7.863135   
       1698387  22878661     36  36_2099_96      7.863135         7.863135   
       1698388  22878663     36  36_2099_96      7.863135         7.863135   
       1698389  22878659     36  36_2099_97      9.718494         9.718494   
       1698390  22878661     36  36_2099_97      9.718494         9.718494   
       1698391  22878663     36  36_2099_97      9.718494         9.718494   
       1698392  22878659     36  36_2099_98     10.096290        10.096290   
       1698393  22878661     36  36_2099_98     10.096290        10.096290   
       1698394  22878663     36  36_2099_98     10.096290        10.096290   
       1698395  22878659     36  36_2099_99     10.749942        10.749942   
       1698396  22878661     36  36_2099_99     10.749942        10.749942   
       1698397  22878663     36  36_2099_99     10.749942        10.749942   

                cov.air_temp_3d  cov.air_temp_6d  cov.air_temp_9d  \
GCM                                                                 
MIROC5 1698368         5.769205         6.509204         6.853404   
       1698369         5.769205         6.509204         6.853404   
       1698370         5.769205         6.509204         6.853404   
       1698371         5.655558         6.720688         6.965672   
       1698372         5.655558         6.720688         6.965672   
       1698373         5.655558         6.720688         6.965672   
       1698374         7.664080         7.358418         7.210713   
       1698375         7.664080         7.358418         7.210713   
       1698376         7.664080         7.358418         7.210713   
       1698377         9.761184         7.765195         7.593197   
       1698378         9.761184         7.765195         7.593197   
       1698379         9.761184         7.765195         7.593197   
       1698380         9.845648         7.750603         7.762341   
       1698381         9.845648         7.750603         7.762341   
       1698382         9.845648         7.750603         7.762341   
       1698383         8.600642         8.132361         7.772492   
       1698384         8.600642         8.132361         7.772492   
       1698385         8.600642         8.132361         7.772492   
       1698386         7.589282         8.675233         7.706557   
       1698387         7.589282         8.675233         7.706557   
    

In [38]:
df.columns

Index(['COMID', 'SegID', 'lookup', 'cov.air_temp', 'cov.air_temp_ws',
       'cov.air_temp_3d', 'cov.air_temp_6d', 'cov.air_temp_9d',
       'cov.air_temp_12d', 'cov.air_temp_15d', 'cov.air_temp_18d',
       'cov.air_temp_21d', 'cov.air_temp_24d', 'cov.air_temp_27d',
       'cov.air_temp_30d', 'cov.air_temp_35d', 'cov.air_temp_40d',
       'cov.air_temp_50d', 'cov.air_temp_60d', 'cov.SWE', 'cov.SWE_ws',
       'cov.Flow', 'cov.Flow_log', 'tim.year', 'tim.doy', 'tim.date',
       'cov.daylength', 'cov.daylength_hours', 'cov.SWE_mean_year',
       'cov.SWE_1Apr', 'cov.SWE_last_doy'],
      dtype='object')

In [39]:
df.tail()

COMID  SegID      lookup  cov.air_temp  cov.air_temp_ws  \
GCM                                                                          
MIROC5 1698393  22878661     36  36_2099_98     10.096290        10.096290   
       1698394  22878663     36  36_2099_98     10.096290        10.096290   
       1698395  22878659     36  36_2099_99     10.749942        10.749942   
       1698396  22878661     36  36_2099_99     10.749942        10.749942   
       1698397  22878663     36  36_2099_99     10.749942        10.749942   

                cov.air_temp_3d  cov.air_temp_6d  cov.air_temp_9d  \
GCM                                                                 
MIROC5 1698393         9.225973         8.913307         8.496898   
       1698394         9.225973         8.913307         8.496898   
       1698395        10.188242         8.888762         9.179569   
       1698396        10.188242         8.888762         9.179569   
       1698397        10.188242         8.888762         9.179569   

                cov.air_temp_12d  cov.air_temp_15d  ...  cov.Flow  \
GCM                                                 ...             
MIROC5 1698393          8.135863          7.891751  ...  8.175002   
       1698394          8.135863          7.891751  ...  8.175002   
       1698395          8.326978          8.111423  ...  8.277249   
       1698396          8.326978          8.111423  ...  8.277249   
       1698397          8.326978          8.111423  ...  8.277249   

                cov.Flow_log  tim.year  tim.doy  tim.date  cov.daylength  \
GCM                                                                        
MIROC5 1698393      2.101081    2099.0     98.0   47214.0        47001.6   
       1698394      2.101081    2099.0     98.0   47214.0        47001.6   
       1698395      2.113511    2099.0     99.0   47215.0        47001.6   
       1698396      2.113511    2099.0     99.0   47215.0        47001.6   
       1698397      2.113511    2099.0     99.0   47215.0        47001.6   

                cov.daylength_hours  cov.SWE_mean_year  cov.SWE_1Apr  \
GCM                                                                    
MIROC5 1698393               13.056          33.657048     13.664044   
       1698394               13.056          33.657048     13.664044   
       1698395               13.056          33.657048     13.664044   
       1698396               13.056          33.657048     13.664044   
       1698397               13.056          33.657048     13.664044   

                cov.SWE_last_doy  
GCM                               
MIROC5 1698393             365.0  
       1698394             365.0  
       1698395             365.0  
       1698396             365.0  
       1698397             365.0  

[5 rows x 31 columns]

In [250]:
%%time
gcm_frames = []
fail_list = []
for cur_gcm in gcms:
    try: gcm_frames.append(ro.r(f"read.fst('/Volumes/Elements/URycki/temp_data/covariates_GCM/{cur_gcm}/huc_{cur_huc}.fst')"))
    except: fail_list.append(cur_gcm)

df = pd.concat(gcm_frames, keys = gcms, names = ['GCM'])
df = df.reset_index().drop('level_1', axis = 1)

df['date'] = pd.to_datetime(df['tim.year'] * 1000 + df['tim.doy'], format='%Y%j')    

<timed exec>:7: FutureWarning: The behavior of pd.concat with len(keys) != len(objs) is deprecated. In a future version this will raise instead of truncating to the smaller of the two sequences


CPU times: user 2min 47s, sys: 1min 57s, total: 4min 45s
Wall time: 1min 31s


In [252]:
df

COMID  SegID      lookup  cov.air_temp  \
GCM                                                               
CanESM2      1        22878641     35   35_1950_1    -23.876358   
             2        22878643     35   35_1950_1    -23.876358   
             3        22878645     35   35_1950_1    -23.876358   
             4        22878647     35   35_1950_1    -23.876358   
             5        22878649     35   35_1950_1    -23.876358   
...                        ...    ...         ...           ...   
IPSL-CM5A-MR 1698393  22878661     36  36_2099_98     10.096290   
             1698394  22878663     36  36_2099_98     10.096290   
             1698395  22878659     36  36_2099_99     10.749942   
             1698396  22878661     36  36_2099_99     10.749942   
             1698397  22878663     36  36_2099_99     10.749942   

                      cov.air_temp_ws  cov.air_temp_3d  cov.air_temp_6d  \
GCM                                                                       
CanESM2      1             -23.677581              NaN              NaN   
             2             -23.677581              NaN              NaN   
             3             -23.677581              NaN              NaN   
             4             -23.677581              NaN              NaN   
             5             -23.677581              NaN              NaN   
...                               ...              ...              ...   
IPSL-CM5A-MR 1698393        10.096290         9.225973         8.913307   
             1698394        10.096290         9.225973         8.913307   
             1698395        10.749942        10.188242         8.888762   
             1698396        10.749942        10.188242         8.888762   
             1698397        10.749942        10.188242         8.888762   

                      cov.air_temp_9d  cov.air_temp_12d  cov.air_temp_15d  \
GCM                                                                         
CanESM2      1                    NaN               NaN               NaN   
             2                    NaN               NaN               NaN   
             3                    NaN               NaN               NaN   
             4                    NaN               NaN               NaN   
             5                    NaN               NaN               NaN   
...                               ...               ...               ...   
IPSL-CM5A-MR 1698393         8.496898          8.135863          7.891751   
             1698394         8.496898          8.135863          7.891751   
             1698395         9.179569          8.326978          8.111423   
             1698396         9.179569          8.326978          8.111423   
             1698397         9.179569          8.326978          8.111423   

                      ...  cov.Flow_log  tim.year  tim.doy  tim.date  \
GCM                   ...                                              
CanESM2      1        ...      0.500972    1950.0      1.0   -7305.0   
             2        ...      0.500972    1950.0      1.0   -7305.0   
             3        ...      0.500972    1950.0      1.0   -7305.0   
             4        ...      0.500972    1950.0      1.0   -7305.0   
             5        ...      0.500972    1950.0      1.0   -7305.0   
...                   ...           ...       ...      ...       ...   
IPSL-CM5A-MR 1698393  ...      2.101081    2099.0     98.0   47214.0   
             1698394  ...      2.101081    2099.0     98.0   47214.0   
             1698395  ...      2.113511    2099.0     99.0   47215.0   
             1698396  ...      2.113511    2099.0     99.0   47215.0   
             1698397  ...      2.113511    2099.0     99.0   47215.0   

                      cov.daylength  cov.daylength_hours  cov.SWE_mean_year  \
GCM                                                                           
CanESM2      1              29376.0                8.160          68.982662   
 

In [250]:
%%time
gcm_frames = []
fail_list = []
for cur_gcm in gcms:
    try: gcm_frames.append(ro.r(f"read.fst('/Volumes/Elements/URycki/temp_data/covariates_GCM/{cur_gcm}/huc_{cur_huc}.fst')"))
    except: fail_list.append(cur_gcm)

df = pd.concat(gcm_frames, keys = gcms, names = ['GCM'])
df = df.reset_index().drop('level_1', axis = 1)

df['date'] = pd.to_datetime(df['tim.year'] * 1000 + df['tim.doy'], format='%Y%j')    

<timed exec>:7: FutureWarning: The behavior of pd.concat with len(keys) != len(objs) is deprecated. In a future version this will raise instead of truncating to the smaller of the two sequences


CPU times: user 2min 47s, sys: 1min 57s, total: 4min 45s
Wall time: 1min 31s


In [45]:
    cur_huc_anoms = []
    cur_comids = list(df.COMID.unique())
    len(cur_comids)

31

In [46]:
df.columns

Index(['COMID', 'SegID', 'lookup', 'cov.air_temp', 'cov.air_temp_ws',
       'cov.air_temp_3d', 'cov.air_temp_6d', 'cov.air_temp_9d',
       'cov.air_temp_12d', 'cov.air_temp_15d', 'cov.air_temp_18d',
       'cov.air_temp_21d', 'cov.air_temp_24d', 'cov.air_temp_27d',
       'cov.air_temp_30d', 'cov.air_temp_35d', 'cov.air_temp_40d',
       'cov.air_temp_50d', 'cov.air_temp_60d', 'cov.SWE', 'cov.SWE_ws',
       'cov.Flow', 'cov.Flow_log', 'tim.year', 'tim.doy', 'tim.date',
       'cov.daylength', 'cov.daylength_hours', 'cov.SWE_mean_year',
       'cov.SWE_1Apr', 'cov.SWE_last_doy', 'date'],
      dtype='object')

In [47]:
df = df.reset_index().drop('level_1', axis = 1)


In [48]:
df

,GCM,COMID,SegID,lookup,cov.air_temp,cov.air_temp_ws,cov.air_temp_3d,cov.air_temp_6d,cov.air_temp_9d,cov.air_temp_12d,...,cov.Flow_log,tim.year,tim.doy,tim.date,cov.daylength,cov.daylength_hours,cov.SWE_mean_year,cov.SWE_1Apr,cov.SWE_last_doy,date
0,CanESM2,22878641,35,35_1950_1,-23.876358,-23.677581,NaN,NaN,NaN,NaN,...,0.500972,1950.0,1.0,-7305.0,29376.0,8.160,68.982662,177.266990,365.0,1950-01-01
1,CanESM2,22878643,35,35_1950_1,-23.876358,-23.677581,NaN,NaN,NaN,NaN,...,0.500972,1950.0,1.0,-7305.0,29376.0,8.160,68.982662,177.266990,365.0,1950-01-01
2,CanESM2,22878645,35,35_1950_1,-23.876358,-23.677581,NaN,NaN,NaN,NaN,...,0.500972,1950.0,1.0,-7305.0,29376.0,8.160,68.982662,177.266990,365.0,1950-01-01
3,CanESM2,22878647,35,35_1950_1,-23.876358,-23.677581,NaN,NaN,NaN,NaN,...,0.500972,1950.0,1.0,-7305.0,29376.0,8.160,68.982662,177.266990,365.0,1950-01-01
4,CanESM2,22878649,35,35_1950_1,-23.876358,-23.677581,NaN,NaN,NaN,NaN,...,0.500972,1950.0,1.0,-7305.0,29376.0,8.160,68.982662,177.266990,365.0,1950-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16983965,MIROC5,22878661,36,36_2099_98,10.096290,10.096290,9.225973,8.913307,8.496898,8.135863,...,2.101081,2099.0,98.0,47214.0,47001.6,13.056,33.657048,13.664044,365.0,2099-04-08
16983966,MIROC5,22878663,36,36_2099_98,10.096290,10.096290,9.225973,8.913307,8.496898,8.135863,...,2.101081,2099.0,98.0,47214.0,47001.6,13.056,33.657048,13.664044,365.0,2099-04-08
16983967,MIROC5,22878659,36,36_2099_99,10.749942,10.749942,10.188242,8.888762,9.179569,8.326978,...,2.113511,2099.0,99.0,47215.0,47001.6,13.056,33.657048,13.664044,365.0,2099-04-09
16983968,MIROC5,22878661,36,36_2099_99,10.749942,10.749942,10.188242,8.888762,9.179569,8.326978,...,2.113511,2099.0,99.0,47215.0,47001.6,13.056,33.657048,13.664044,365.0,2099-04-09


In [29]:
cols = ['GCM', 'COMID', 'date', 'cov.air_temp_ws', 'cov.Flow_log', 'cov.SWE_ws', 'tim.doy']

In [79]:
    for j in [cur_comids[0]]:
        print(j)
        cur_preds = df[df.COMID == j][cols]
        cur_ens_med = cur_preds.drop('GCM', axis = 1).groupby('date').median()
        #cur_ens_med['doy'] = cur_ens_med.index.dayofyear
        cur_ens_med['season'] = cur_ens_med['tim.doy'].apply(assign_season)

22878641


In [30]:
periods
periods_idx = list(periods.keys())
periods_idx

['2020s', '1990s', '2050s', '2080s']

In [70]:
cur_ens_med

,COMID,cov.air_temp_ws,cov.Flow_log,cov.SWE_mean_year,tim.doy,season
date,,,,,,
1950-01-01,22878641.0,-4.155098,0.504667,64.149632,1.0,winter
1950-01-02,22878641.0,-5.742823,0.995216,64.149632,2.0,winter
1950-01-03,22878641.0,-5.751736,1.069237,64.149632,3.0,winter
1950-01-04,22878641.0,-7.751066,1.072651,64.149632,4.0,winter
1950-01-05,22878641.0,-8.956133,1.074674,64.149632,5.0,winter
...,...,...,...,...,...,...
2099-12-27,22878641.0,-0.058756,1.379495,15.230097,361.0,winter
2099-12-28,22878641.0,-0.844916,1.222724,15.230097,362.0,winter
2099-12-29,22878641.0,-0.995410,0.974905,15.230097,363.0,winter


In [71]:
    for j in [cur_comids[0]]:
        print(j)
        cur_preds = df[df.COMID == j]
        cur_ens_med = cur_preds.drop(['lookup', 'GCM', 'SegID', 'tim.year', 'tim.date'], axis = 1).groupby('date').median()
        #cur_ens_med['doy'] = cur_ens_med.index.dayofyear
        cur_ens_med['season'] = cur_ens_med['tim.doy'].apply(assign_season)

        cur_anomolies = []
        for dec in periods_idx:
            if dec == '2020s': 
                cur = cur_ens_med[cur_ens_med.index.year.isin(periods[dec])].groupby('season').median()#['prd.stream_temp']
                cur_anomolies.append(cur)
            else: cur_anomolies.append(cur_ens_med[cur_ens_med.index.year.isin(periods[dec])].groupby('season').median().subtract(cur))
        cur_seas_anomolies = pd.concat(cur_anomolies, axis = 1, keys = periods_idx)

22878641


In [72]:
cur_seas_anomolies

2020s                                               \
             COMID cov.air_temp cov.air_temp_ws cov.air_temp_3d   
season                                                            
fall    22878641.0     0.677336        0.239469        0.932781   
spring  22878641.0     8.140971        7.696518        8.019155   
summer  22878641.0    16.508109       16.092271       16.472997   
winter  22878641.0    -2.498689       -2.933752       -2.852580   

                                                                          \
       cov.air_temp_6d cov.air_temp_9d cov.air_temp_12d cov.air_temp_15d   
season                                                                     
fall          1.109749        1.236818         1.618643         2.060728   
spring        7.846902        7.669322         7.518496         7.254511   
summer       16.461175       16.461796        16.490585        16.540808   
winter       -3.142961       -3.427111        -3.717662        -3.929594   

                                          ...      2080s                       \
       cov.air_temp_18d cov.air_temp_21d  ...    cov.SWE cov.SWE_ws  cov.Flow   
season                                    ...                                   
fall           2.427483         2.790648  ...  -3.784594  -2.011112 -0.245415   
spring         6.926886         6.825393  ... -60.693358 -51.179166 -1.654567   
summer        16.497536        16.482803  ...   0.000000   0.000000 -1.047490   
winter        -4.121731        -4.314990  ... -60.307974 -87.219756  2.788734   

                                                               \
       cov.Flow_log tim.doy cov.daylength cov.daylength_hours   
season                                                          
fall      -0.186106     0.0           0.0                 0.0   
spring    -0.176655     0.0           0.0                 0.0   
summer    -0.431228     0.0           0.0                 0.0   
winter     0.936096     0.0           0.0                 0.0   

                                                        
       cov.SWE_mean_year cov.SWE_1Apr cov.SWE_last_doy  
season                                                  
fall          -40.261129  -149.947877              0.0  
spring        -40.261129  -149.947877              0.0  
summer        -40.261129  -149.947877              0.0  
winter        -38.414710  -129.685742              0.0  

[4 rows x 108 columns]

In [73]:
cur_ens_med

,COMID,cov.air_temp,cov.air_temp_ws,cov.air_temp_3d,cov.air_temp_6d,cov.air_temp_9d,cov.air_temp_12d,cov.air_temp_15d,cov.air_temp_18d,cov.air_temp_21d,...,cov.SWE_ws,cov.Flow,cov.Flow_log,tim.doy,cov.daylength,cov.daylength_hours,cov.SWE_mean_year,cov.SWE_1Apr,cov.SWE_last_doy,season
date,,,,,,,,,,,,,,,,,,,,,
1950-01-01,22878641.0,-3.673526,-4.155098,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,96.598599,1.656434,0.504667,1.0,29376.0,8.160,64.149632,186.349027,365.0,winter
1950-01-02,22878641.0,-5.399029,-5.742823,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,99.890183,2.705316,0.995216,2.0,29376.0,8.160,64.149632,186.349027,365.0,winter
1950-01-03,22878641.0,-5.448190,-5.751736,-4.564538,NaN,NaN,NaN,NaN,NaN,NaN,...,106.197209,2.913166,1.069237,3.0,29376.0,8.160,64.149632,186.349027,365.0,winter
1950-01-04,22878641.0,-7.401785,-7.751066,-5.664489,NaN,NaN,NaN,NaN,NaN,NaN,...,107.292981,2.923119,1.072651,4.0,29376.0,8.160,64.149632,186.349027,365.0,winter
1950-01-05,22878641.0,-8.607928,-8.956133,-7.126086,NaN,NaN,NaN,NaN,NaN,NaN,...,107.388125,2.929038,1.074674,5.0,29721.6,8.256,64.149632,186.349027,365.0,winter
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2099-12-27,22878641.0,0.666437,-0.058756,1.536446,0.838072,1.349781,1.150979,1.441929,1.436148,1.624903,...,15.626225,4.100803,1.379495,361.0,29030.4,8.064,15.230097,24.268455,365.0,winter
2099-12-28,22878641.0,-0.432673,-0.844916,0.252072,0.809195,1.245946,0.946475,1.160170,1.389482,1.503036,...,14.873036,3.619834,1.222724,362.0,29030.4,8.064,15.230097,24.268455,365.0,winter
2099-12-29,22878641.0,-0.549001,-0.995410,-0.467249,0.349452,0.791587,0.936249,1.048310,1.340706,1.388432,...,14.596719,2.776473,0.974905,363.0,29030.4,8.064,15.230097,24.268455,365.0,winter


In [311]:
pd.concat(cur_anomolies, axis = 1, keys = periods_idx)

2020s                                               \
             COMID cov.air_temp cov.air_temp_ws cov.air_temp_3d   
season                                                            
fall    22878701.0     0.014692       -0.175033        0.207123   
spring  22878701.0     7.653825        7.575026        7.524950   
summer  22878701.0    15.990297       15.850337       15.870330   
winter  22878701.0    -3.167194       -3.219386       -3.377902   

                                                                          \
       cov.air_temp_6d cov.air_temp_9d cov.air_temp_12d cov.air_temp_15d   
season                                                                     
fall          0.390594        0.725011         1.034103         1.374594   
spring        7.198784        7.125863         6.971728         6.695704   
summer       15.885192       15.994240        15.969507        15.986753   
winter       -3.678337       -3.938703        -4.209577        -4.360757   

                                          ...      2080s                       \
       cov.air_temp_18d cov.air_temp_21d  ...    cov.SWE cov.SWE_ws  cov.Flow   
season                                    ...                                   
fall           1.691612         2.145506  ...  -1.503527  -1.107441 -0.047153   
spring         6.447333         6.237052  ... -61.986096 -50.041769 -0.984029   
summer        15.940827        15.839691  ...   0.000000   0.000000 -0.669396   
winter        -4.664209        -4.762205  ... -81.972680 -96.526663  2.212939   

                                                               \
       cov.Flow_log tim.doy cov.daylength cov.daylength_hours   
season                                                          
fall      -0.053813     0.0           0.0                 0.0   
spring    -0.145776     0.0           0.0                 0.0   
summer    -0.454083     0.0           0.0                 0.0   
winter     1.175515     0.0           0.0                 0.0   

                                                        
       cov.SWE_mean_year cov.SWE_1Apr cov.SWE_last_doy  
season                                                  
fall          -44.424592  -178.280072              0.0  
spring        -44.424592  -178.280072              0.0  
summer        -44.424592  -178.280072              0.0  
winter        -40.171856  -158.760089              0.0  

[4 rows x 108 columns]

In [74]:
    for j in [cur_comids[0]]:
        print(j)
        cur_preds = df[df.COMID == j]
        cur_ens_med = cur_preds.drop(['lookup', 'GCM', 'SegID', 'tim.year', 'tim.date'], axis = 1).groupby('date').median()
        #cur_ens_med['doy'] = cur_ens_med.index.dayofyear
        cur_ens_med['season'] = cur_ens_med['tim.doy'].apply(assign_season)

        cur_anomolies = {}
        for dec in list(periods.keys()):
            if dec == '2020s': cur_anomolies[dec] = cur_ens_med[cur_ens_med.index.year.isin(periods[dec])].groupby('season').median()#['prd.stream_temp']
            #else: cur_anomolies[dec] = cur_ens_med[cur_ens_med.index.year.isin(periods[dec])].groupby('season').median()['prd.stream_temp'].subtract(cur_anomolies['2020s'])
            else: cur_anomolies[dec] = cur_ens_med[cur_ens_med.index.year.isin(periods[dec])].groupby('season').median().subtract(cur_anomolies['2020s'])
        cur_seas_anomolies = pd.DataFrame(cur_anomolies)    

22878641


ValueError: If using all scalar values, you must pass an index

In [63]:
df


,GCM,COMID,SegID,lookup,cov.air_temp,cov.air_temp_ws,cov.air_temp_3d,cov.air_temp_6d,cov.air_temp_9d,cov.air_temp_12d,...,cov.Flow_log,tim.year,tim.doy,tim.date,cov.daylength,cov.daylength_hours,cov.SWE_mean_year,cov.SWE_1Apr,cov.SWE_last_doy,date
0,CanESM2,22878641,35,35_1950_1,-23.876358,-23.677581,NaN,NaN,NaN,NaN,...,0.500972,1950.0,1.0,-7305.0,29376.0,8.160,68.982662,177.266990,365.0,1950-01-01
1,CanESM2,22878643,35,35_1950_1,-23.876358,-23.677581,NaN,NaN,NaN,NaN,...,0.500972,1950.0,1.0,-7305.0,29376.0,8.160,68.982662,177.266990,365.0,1950-01-01
2,CanESM2,22878645,35,35_1950_1,-23.876358,-23.677581,NaN,NaN,NaN,NaN,...,0.500972,1950.0,1.0,-7305.0,29376.0,8.160,68.982662,177.266990,365.0,1950-01-01
3,CanESM2,22878647,35,35_1950_1,-23.876358,-23.677581,NaN,NaN,NaN,NaN,...,0.500972,1950.0,1.0,-7305.0,29376.0,8.160,68.982662,177.266990,365.0,1950-01-01
4,CanESM2,22878649,35,35_1950_1,-23.876358,-23.677581,NaN,NaN,NaN,NaN,...,0.500972,1950.0,1.0,-7305.0,29376.0,8.160,68.982662,177.266990,365.0,1950-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16983965,MIROC5,22878661,36,36_2099_98,10.096290,10.096290,9.225973,8.913307,8.496898,8.135863,...,2.101081,2099.0,98.0,47214.0,47001.6,13.056,33.657048,13.664044,365.0,2099-04-08
16983966,MIROC5,22878663,36,36_2099_98,10.096290,10.096290,9.225973,8.913307,8.496898,8.135863,...,2.101081,2099.0,98.0,47214.0,47001.6,13.056,33.657048,13.664044,365.0,2099-04-08
16983967,MIROC5,22878659,36,36_2099_99,10.749942,10.749942,10.188242,8.888762,9.179569,8.326978,...,2.113511,2099.0,99.0,47215.0,47001.6,13.056,33.657048,13.664044,365.0,2099-04-09
16983968,MIROC5,22878661,36,36_2099_99,10.749942,10.749942,10.188242,8.888762,9.179569,8.326978,...,2.113511,2099.0,99.0,47215.0,47001.6,13.056,33.657048,13.664044,365.0,2099-04-09


In [80]:
%%time
cur_huc_anoms = []
for j in [cur_comids[0]]:
    print(j)
    cur_preds = df[df.COMID == j][cols]
    cur_ens_med = cur_preds.drop('GCM', axis = 1).groupby('date').median()
    #cur_ens_med['doy'] = cur_ens_med.index.dayofyear
    cur_ens_med['season'] = cur_ens_med['tim.doy'].apply(assign_season)

    cur_anomolies = {}
    for dec in list(periods.keys()):
        if dec == '2020s': cur_anomolies[dec] = cur_ens_med[cur_ens_med.index.year.isin(periods[dec])].groupby('season').median()#['prd.stream_temp']
        #else: cur_anomolies[dec] = cur_ens_med[cur_ens_med.index.year.isin(periods[dec])].groupby('season').median()['prd.stream_temp'].subtract(cur_anomolies['2020s'])
        else: cur_anomolies[dec] = cur_ens_med[cur_ens_med.index.year.isin(periods[dec])].groupby('season').median().subtract(cur_anomolies['2020s'])
    cur_seas_anomolies = pd.concat(cur_anomolies, axis = 1, keys = periods_idx)
    #cur_huc_anoms.append(cur_seas_anomolies)

22878641
CPU times: user 141 ms, sys: 7.2 ms, total: 148 ms
Wall time: 143 ms


In [81]:
cur_ens_med

,COMID,cov.air_temp_ws,cov.Flow_log,cov.SWE_ws,tim.doy,season
date,,,,,,
1950-01-01,22878641.0,-4.155098,0.504667,96.598599,1.0,winter
1950-01-02,22878641.0,-5.742823,0.995216,99.890183,2.0,winter
1950-01-03,22878641.0,-5.751736,1.069237,106.197209,3.0,winter
1950-01-04,22878641.0,-7.751066,1.072651,107.292981,4.0,winter
1950-01-05,22878641.0,-8.956133,1.074674,107.388125,5.0,winter
...,...,...,...,...,...,...
2099-12-27,22878641.0,-0.058756,1.379495,15.626225,361.0,winter
2099-12-28,22878641.0,-0.844916,1.222724,14.873036,362.0,winter
2099-12-29,22878641.0,-0.995410,0.974905,14.596719,363.0,winter


In [82]:
cur_seas_anomolies

2020s                                                  1990s  \
             COMID cov.air_temp_ws cov.Flow_log  cov.SWE_ws tim.doy COMID   
season                                                                      
fall    22878641.0        0.239469     0.391416    2.140895   310.0   0.0   
spring  22878641.0        7.696518     2.342339   59.850854   125.5   0.0   
summer  22878641.0       16.092271     1.098443    0.000000   217.5   0.0   
winter  22878641.0       -2.933752     0.556656  161.286491    45.0   0.0   

                                                       2050s                  \
       cov.air_temp_ws cov.Flow_log cov.SWE_ws tim.doy COMID cov.air_temp_ws   
season                                                                         
fall         -0.678125     0.088227   1.635413     0.0   0.0        1.682408   
spring       -1.101534    -0.026383  25.644863     0.0   0.0        1.630595   
summer       -1.812675     0.210804   0.000000     0.0   0.0        2.063963   
winter       -1.318718    -0.181262  13.458769     0.0   0.0        0.754254   

                                       2080s                               \
       cov.Flow_log cov.SWE_ws tim.doy COMID cov.air_temp_ws cov.Flow_log   
season                                                                      
fall      -0.193711  -1.331763     0.0   0.0        3.540243    -0.186106   
spring    -0.108620 -33.748901     0.0   0.0        3.120149    -0.176655   
summer    -0.252313   0.000000     0.0   0.0        5.047131    -0.431228   
winter     0.243281 -41.294432     0.0   0.0        2.999358     0.936096   

                           
       cov.SWE_ws tim.doy  
season                     
fall    -2.011112     0.0  
spring -51.179166     0.0  
summer   0.000000     0.0  
winter -87.219756     0.0

In [85]:
%%time
cur_huc_anoms = []
for j in cur_comids:
    print(j)
    cur_preds = df[df.COMID == j][cols]
    cur_ens_med = cur_preds.drop('GCM', axis = 1).groupby('date').median()
    #cur_ens_med['doy'] = cur_ens_med.index.dayofyear
    cur_ens_med['season'] = cur_ens_med['tim.doy'].apply(assign_season)

    cur_anomolies = []
    for dec in periods_idx:
        if dec == '2020s': 
            cur = cur_ens_med[cur_ens_med.index.year.isin(periods[dec])].groupby('season').median()#['prd.stream_temp']
            cur_anomolies.append(cur)
        else: cur_anomolies.append(cur_ens_med[cur_ens_med.index.year.isin(periods[dec])].groupby('season').median().subtract(cur))
    cur_seas_anomolies = pd.concat(cur_anomolies, axis = 1, keys = periods_idx)
    cur_huc_anoms.append(cur_seas_anomolies)

22878641
22878643
22878645
22878647
22878649
22878651
22878653
22878655
22878657
22878743
22878745
22878747
22878749
22878751
22878753
22878755
22878757
22878759
22878761
22879621
22878665
22878667
22878699
22878701
22878703
22878705
22878707
22878709
22878659
22878661
22878663
CPU times: user 3.36 s, sys: 39.7 ms, total: 3.4 s
Wall time: 3.39 s


In [ ]:
cur

In [86]:
cur_huc_anoms

[             2020s                                                  1990s  \
              COMID cov.air_temp_ws cov.Flow_log  cov.SWE_ws tim.doy COMID   
 season                                                                      
 fall    22878641.0        0.239469     0.391416    2.140895   310.0   0.0   
 spring  22878641.0        7.696518     2.342339   59.850854   125.5   0.0   
 summer  22878641.0       16.092271     1.098443    0.000000   217.5   0.0   
 winter  22878641.0       -2.933752     0.556656  161.286491    45.0   0.0   
 
                                                        2050s                  \
        cov.air_temp_ws cov.Flow_log cov.SWE_ws tim.doy COMID cov.air_temp_ws   
 season                                                                         
 fall         -0.678125     0.088227   1.635413     0.0   0.0        1.682408   
 spring       -1.101534    -0.026383  25.644863     0.0   0.0        1.630595   
 summer       -1.812675     0.210804   0.000000

In [353]:
df

,GCM,COMID,SegID,lookup,cov.air_temp,cov.air_temp_ws,cov.air_temp_3d,cov.air_temp_6d,cov.air_temp_9d,cov.air_temp_12d,...,cov.Flow_log,tim.year,tim.doy,tim.date,cov.daylength,cov.daylength_hours,cov.SWE_mean_year,cov.SWE_1Apr,cov.SWE_last_doy,date
0,CanESM2,22878641,35,35_1950_1,-23.876358,-23.677581,NaN,NaN,NaN,NaN,...,0.500972,1950.0,1.0,-7305.0,29376.0,8.160,68.982662,177.266990,365.0,1950-01-01
1,CanESM2,22878643,35,35_1950_1,-23.876358,-23.677581,NaN,NaN,NaN,NaN,...,0.500972,1950.0,1.0,-7305.0,29376.0,8.160,68.982662,177.266990,365.0,1950-01-01
2,CanESM2,22878645,35,35_1950_1,-23.876358,-23.677581,NaN,NaN,NaN,NaN,...,0.500972,1950.0,1.0,-7305.0,29376.0,8.160,68.982662,177.266990,365.0,1950-01-01
3,CanESM2,22878647,35,35_1950_1,-23.876358,-23.677581,NaN,NaN,NaN,NaN,...,0.500972,1950.0,1.0,-7305.0,29376.0,8.160,68.982662,177.266990,365.0,1950-01-01
4,CanESM2,22878649,35,35_1950_1,-23.876358,-23.677581,NaN,NaN,NaN,NaN,...,0.500972,1950.0,1.0,-7305.0,29376.0,8.160,68.982662,177.266990,365.0,1950-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15285568,IPSL-CM5A-MR,22878661,36,36_2099_98,10.096290,10.096290,9.225973,8.913307,8.496898,8.135863,...,2.101081,2099.0,98.0,47214.0,47001.6,13.056,33.657048,13.664044,365.0,2099-04-08
15285569,IPSL-CM5A-MR,22878663,36,36_2099_98,10.096290,10.096290,9.225973,8.913307,8.496898,8.135863,...,2.101081,2099.0,98.0,47214.0,47001.6,13.056,33.657048,13.664044,365.0,2099-04-08
15285570,IPSL-CM5A-MR,22878659,36,36_2099_99,10.749942,10.749942,10.188242,8.888762,9.179569,8.326978,...,2.113511,2099.0,99.0,47215.0,47001.6,13.056,33.657048,13.664044,365.0,2099-04-09
15285571,IPSL-CM5A-MR,22878661,36,36_2099_99,10.749942,10.749942,10.188242,8.888762,9.179569,8.326978,...,2.113511,2099.0,99.0,47215.0,47001.6,13.056,33.657048,13.664044,365.0,2099-04-09


In [392]:
%%time
cur_huc_anoms = []
for j in cur_comids:
    print(j)
    cur_preds = df[df.COMID == j]
    cur_ens_med = cur_preds.drop(['lookup', 'GCM', 'SegID', 'tim.year', 'tim.date'], axis = 1).groupby('date').median()
    #cur_ens_med['doy'] = cur_ens_med.index.dayofyear
    cur_ens_med['season'] = cur_ens_med['tim.doy'].apply(assign_season)

    cur_anomolies = []
    cur = cur_ens_med[cur_ens_med.index.year.isin(periods['2020s'])].groupby('season').median()
    for dec in periods_idx:
        if dec == '2020s': cur_anomolies.append(cur.drop(['COMID', 'tim.doy'], axis = 1))
        else: cur_anomolies.append(cur_ens_med[cur_ens_med.index.year.isin(periods[dec])].groupby('season').median().subtract(cur).drop(['COMID', 'tim.doy'], axis = 1))
    cur_seas_anomolies = pd.concat(cur_anomolies, axis = 1, keys = periods_idx)
    cur_huc_anoms.append(cur_seas_anomolies)

22878641
22878643
22878645
22878647
22878649
22878651
22878653
22878655
22878657
22878743
22878745
22878747
22878749
22878751
22878753
22878755
22878757
22878759
22878761
22879621
22878665
22878667
22878699
22878701
22878703
22878705
22878707
22878709
22878659
22878661
22878663
CPU times: user 4.56 s, sys: 411 ms, total: 4.97 s
Wall time: 5 s


In [394]:
cur_preds['cov.SWE_last_doy'].unique().min()

np.float64(357.0)

In [377]:
cur_preds.columns

Index(['GCM', 'COMID', 'SegID', 'lookup', 'cov.air_temp', 'cov.air_temp_ws',
       'cov.air_temp_3d', 'cov.air_temp_6d', 'cov.air_temp_9d',
       'cov.air_temp_12d', 'cov.air_temp_15d', 'cov.air_temp_18d',
       'cov.air_temp_21d', 'cov.air_temp_24d', 'cov.air_temp_27d',
       'cov.air_temp_30d', 'cov.air_temp_35d', 'cov.air_temp_40d',
       'cov.air_temp_50d', 'cov.air_temp_60d', 'cov.SWE', 'cov.SWE_ws',
       'cov.Flow', 'cov.Flow_log', 'tim.year', 'tim.doy', 'tim.date',
       'cov.daylength', 'cov.daylength_hours', 'cov.SWE_mean_year',
       'cov.SWE_1Apr', 'cov.SWE_last_doy', 'date'],
      dtype='object')

In [379]:
covariate_metadata.COVARIATE.values

array(['COMID', 'cov.antec_air_temp', 'cov.air_temp_mean_ws',
       'cov.SWE_ws', 'cov.daylength_hours', 'tim.doy',
       'cov.std_mean_flow', 'cov.SWE_1Apr', 'cov.lat_v',
       'cov.elev_mean_smo', 'cov.elev_diff', 'cov.area_km2_ws_log',
       'cov.BFI_cat', 'cov.slope', 'cov.pct_ow_ws', 'cov.pct_wet_all_ws',
       'cov.pct_ice_ws', 'cov.pct_for_all_cat_rip100m', 'cov.canopy_line',
       'cov.pct_urb_all_ws', 'cov.pct_extru_vol_ws', 'cov.precip_cat',
       nan, 'cov.air_temp_range_cat', 'cov.air_temp_mean_cat',
       'cov.air_temp_min_cat', 'cov.air_temp_min_ws', 'cov.area_km2_ws',
       'cov.area_sqkm', 'cov.BFI_ws', 'cov.canal_dens_cat',
       'cov.canal_dens_ws', 'cov.dam_dens_cat', 'cov.dam_dens_ws',
       'cov.dam_hgt_m', 'cov.dam_main_use', 'cov.dam_name',
       'cov.dam_nrm_stor_cat', 'cov.dam_nrm_stor_ws',
       'cov.distance_below_dam', 'cov.elev_max_smo', 'cov.elev_min_smo',
       'cov.elev_cat', 'cov.elev_ws', 'cov.length_km', 'cov.om_cat',
       'cov.om_ws',

In [391]:
undef_list = list(set(cur_preds.columns).difference(set(covariate_metadata.COVARIATE)))
undef = [str(i) for i in undef_list]
undef.sort()
undef

['GCM',
 'SegID',
 'cov.Flow',
 'cov.Flow_log',
 'cov.SWE',
 'cov.SWE_last_doy',
 'cov.SWE_mean_year',
 'cov.air_temp',
 'cov.air_temp_12d',
 'cov.air_temp_15d',
 'cov.air_temp_18d',
 'cov.air_temp_21d',
 'cov.air_temp_24d',
 'cov.air_temp_27d',
 'cov.air_temp_30d',
 'cov.air_temp_35d',
 'cov.air_temp_3d',
 'cov.air_temp_40d',
 'cov.air_temp_50d',
 'cov.air_temp_60d',
 'cov.air_temp_6d',
 'cov.air_temp_9d',
 'cov.air_temp_ws',
 'cov.daylength',
 'date',
 'lookup',
 'tim.date',
 'tim.year']

In [393]:
huc_anoms = pd.concat(cur_huc_anoms, keys = cur_comids)
huc_anoms = huc_anoms.reset_index(names = ['COMID', 'season'])
huc_anoms

COMID  season        2020s                                  \
                      cov.air_temp cov.air_temp_ws cov.air_temp_3d   
0    22878641    fall     0.628859        0.129601        0.831246   
1    22878641  spring     8.227720        7.791607        8.095319   
2    22878641  summer    16.535612       16.108717       16.471788   
3    22878641  winter    -2.423634       -2.954480       -2.748495   
4    22878643    fall     0.628859        0.129601        0.831246   
..        ...     ...          ...             ...             ...   
119  22878661  winter    -3.227137       -3.227137       -3.508586   
120  22878663    fall    -0.205160       -0.205160        0.113062   
121  22878663  spring     7.390042        7.390042        7.321545   
122  22878663  summer    15.826639       15.826639       15.724772   
123  22878663  winter    -3.227137       -3.227137       -3.508586   

                                                                       \
    cov.air_temp_6d cov.air_temp_9d cov.air_temp_12d cov.air_temp_15d   
0          0.975581        1.247479         1.468359         1.876361   
1          7.784169        7.641089         7.501050         7.256160   
2         16.441164       16.487775        16.555442        16.546236   
3         -3.038248       -3.327229        -3.623662        -3.795583   
4          0.975581        1.247479         1.468359         1.876361   
..              ...             ...              ...              ...   
119       -3.747147       -4.078781        -4.365622        -4.578135   
120        0.258598        0.530620         0.775552         1.170552   
121        7.051854        6.878598         6.788595         6.548631   
122       15.809332       15.820356        15.838543        15.789578   
123       -3.747147       -4.078781        -4.365622        -4.578135   

                      ...            2080s                                    \
    cov.air_temp_18d  ... cov.air_temp_60d     cov.SWE  cov.SWE_ws  cov.Flow   
0           2.254910  ...         4.013769   -3.556974   -2.120079 -0.209053   
1           6.984273  ...         3.227859  -55.932374  -49.693772 -1.266930   
2          16.494406  ...         4.489837    0.000000    0.000000 -0.984713   
3          -4.144926  ...         3.720792  -54.796738  -80.074447  2.753776   
4           2.254910  ...         4.013769   -3.556974   -2.120079 -0.209053   
..               ...  ...              ...         ...         ...       ...   
119        -4.804963  ...         3.629946 -100.453806 -100.453806  1.837617   
120         1.545586  ...         3.946368   -0.775128   -0.775128 -0.064575   
121         6.227086  ...         3.184017  -45.279212  -45.279212 -0.815296   
122        15.736803  ...         4.431998    0.000000    0.000000 -0.442362   
123        -4.804963  ...         3.629946 -100.453806 -100.453806  1.837617   

                                                                      \
    cov.Flow_log cov.daylength cov.daylength_hours cov.SWE_mean_year   
0      -0.151465           0.0                 0.0        -37.836670   
1      -0.131862           0.0                 0.0        -37.836670   
2      -0.395023           0.0                 0.0        -37.836670   
3       0.946944           0.0                 0.0        -33.916731   
4      -0.151465           0.0                 0.0        -37.836670   
..           ...           ...                 ...               ...   
119     1.173579           0.0                 0.0        -40.634333   
120    -0.092606           0.0                 0.0        -45.695206   
121    -0.138913           0.0                 0.0        -45.695206   
122    -0.370006           0.0                 0.0        -45.695206   
123     1.173579           0.0                 0.0        -40.634333   

                                   
    cov.SWE_1Apr cov.SWE_last_doy  
0    -144.642370              0.0  
1    -144.642370              0.0  
2    -144.642370         

In [400]:
cur_preds.columns

Index(['GCM', 'COMID', 'SegID', 'lookup', 'cov.air_temp', 'cov.air_temp_ws',
       'cov.air_temp_3d', 'cov.air_temp_6d', 'cov.air_temp_9d',
       'cov.air_temp_12d', 'cov.air_temp_15d', 'cov.air_temp_18d',
       'cov.air_temp_21d', 'cov.air_temp_24d', 'cov.air_temp_27d',
       'cov.air_temp_30d', 'cov.air_temp_35d', 'cov.air_temp_40d',
       'cov.air_temp_50d', 'cov.air_temp_60d', 'cov.SWE', 'cov.SWE_ws',
       'cov.Flow', 'cov.Flow_log', 'tim.year', 'tim.doy', 'tim.date',
       'cov.daylength', 'cov.daylength_hours', 'cov.SWE_mean_year',
       'cov.SWE_1Apr', 'cov.SWE_last_doy', 'date'],
      dtype='object')

In [ ]:
huc_anoms.to_csv(os.path.join(temp_dir_gcms, 'seasonal_anomolies', f'{cur_huc}_anoms.csv'))
with py7zr.SevenZipFile(os.path.join(temp_dir_gcms, 'seasonal_anomolies', f'{cur_huc}_anoms.7z'), 'w') as archive:
    archive.write(os.path.join(temp_dir_gcms, 'seasonal_anomolies', f'{cur_huc}_anoms.csv'))
#os.remove(os.path.join(temp_dir_gcms, 'seasonal_anomolies', f'{cur_huc}_anoms.csv')) 

22878641
22878643
22878645
22878647
22878649
22878651
22878653
22878655
22878657
22878743
22878745
22878747
22878749
22878751
22878753
22878755
22878757
22878759
22878761
22879621
22878665


In [279]:
cur_anomolies

{'2020s':              COMID  cov.air_temp  cov.air_temp_ws  cov.air_temp_3d  \
 season                                                               
 fall    22878701.0      0.014692        -0.175033         0.207123   
 spring  22878701.0      7.653825         7.575026         7.524950   
 summer  22878701.0     15.990297        15.850337        15.870330   
 winter  22878701.0     -3.167194        -3.219386        -3.377902   
 
         cov.air_temp_6d  cov.air_temp_9d  cov.air_temp_12d  cov.air_temp_15d  \
 season                                                                         
 fall           0.390594         0.725011          1.034103          1.374594   
 spring         7.198784         7.125863          6.971728          6.695704   
 summer        15.885192        15.994240         15.969507         15.986753   
 winter        -3.678337        -3.938703         -4.209577         -4.360757   
 
         cov.air_temp_18d  cov.air_temp_21d  ...     cov.SWE  cov.SWE_ws  \

In [294]:
cur_anomolies.keys()

dict_keys(['2020s', '1990s', '2050s', '2080s'])

In [301]:
cur_anomolies['2020s']

,COMID,cov.air_temp,cov.air_temp_ws,cov.air_temp_3d,cov.air_temp_6d,cov.air_temp_9d,cov.air_temp_12d,cov.air_temp_15d,cov.air_temp_18d,cov.air_temp_21d,...,cov.SWE,cov.SWE_ws,cov.Flow,cov.Flow_log,tim.doy,cov.daylength,cov.daylength_hours,cov.SWE_mean_year,cov.SWE_1Apr,cov.SWE_last_doy
season,,,,,,,,,,,,,,,,,,,,,
fall,22878701.0,0.014692,-0.175033,0.207123,0.390594,0.725011,1.034103,1.374594,1.691612,2.145506,...,1.503527,1.107441,0.900032,-0.105325,310.0,33523.2,9.312,68.353887,205.630182,365.0
spring,22878701.0,7.653825,7.575026,7.524950,7.198784,7.125863,6.971728,6.695704,6.447333,6.237052,...,62.281568,50.337241,7.254236,1.981586,125.5,52531.2,14.592,68.353887,205.630182,365.0
summer,22878701.0,15.990297,15.850337,15.870330,15.885192,15.994240,15.969507,15.986753,15.940827,15.839691,...,0.000000,0.000000,1.834110,0.606559,217.5,52704.0,14.640,68.353887,205.630182,365.0
winter,22878701.0,-3.167194,-3.219386,-3.377902,-3.678337,-3.938703,-4.209577,-4.360757,-4.664209,-4.762205,...,160.963813,176.023214,0.988003,-0.012069,45.0,33523.2,9.312,64.222599,186.371076,365.0


In [300]:
pd.DataFrame(cur_anomolies, index = list(cur_anomolies.keys()))

ValueError: Data must be 1-dimensional, got ndarray of shape (4, 27) instead

In [289]:
        cur_seas_anomolies = pd.concat(cur_anomolies.values(), axis = 1)

In [290]:
cur_seas_anomolies

,COMID,cov.air_temp,cov.air_temp_ws,cov.air_temp_3d,cov.air_temp_6d,cov.air_temp_9d,cov.air_temp_12d,cov.air_temp_15d,cov.air_temp_18d,cov.air_temp_21d,...,cov.SWE,cov.SWE_ws,cov.Flow,cov.Flow_log,tim.doy,cov.daylength,cov.daylength_hours,cov.SWE_mean_year,cov.SWE_1Apr,cov.SWE_last_doy
season,,,,,,,,,,,,,,,,,,,,,
fall,22878701.0,0.014692,-0.175033,0.207123,0.390594,0.725011,1.034103,1.374594,1.691612,2.145506,...,-1.503527,-1.107441,-0.047153,-0.053813,0.0,0.0,0.0,-44.424592,-178.280072,0.0
spring,22878701.0,7.653825,7.575026,7.524950,7.198784,7.125863,6.971728,6.695704,6.447333,6.237052,...,-61.986096,-50.041769,-0.984029,-0.145776,0.0,0.0,0.0,-44.424592,-178.280072,0.0
summer,22878701.0,15.990297,15.850337,15.870330,15.885192,15.994240,15.969507,15.986753,15.940827,15.839691,...,0.000000,0.000000,-0.669396,-0.454083,0.0,0.0,0.0,-44.424592,-178.280072,0.0
winter,22878701.0,-3.167194,-3.219386,-3.377902,-3.678337,-3.938703,-4.209577,-4.360757,-4.664209,-4.762205,...,-81.972680,-96.526663,2.212939,1.175515,0.0,0.0,0.0,-40.171856,-158.760089,0.0


In [ ]:
        cur_huc_anoms.append(cur_seas_anomolies)

22878641


ValueError: If using all scalar values, you must pass an index

In [263]:
cur_ens_med

,COMID,SegID,cov.air_temp,cov.air_temp_ws,cov.air_temp_3d,cov.air_temp_6d,cov.air_temp_9d,cov.air_temp_12d,cov.air_temp_15d,cov.air_temp_18d,...,cov.Flow_log,tim.year,tim.doy,tim.date,cov.daylength,cov.daylength_hours,cov.SWE_mean_year,cov.SWE_1Apr,cov.SWE_last_doy,season
date,,,,,,,,,,,,,,,,,,,,,
1950-01-01,22878701.0,36.0,-3.182958,-3.182958,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.075227,1950.0,1.0,-7305.0,29376.0,8.160,69.880874,216.938301,365.0,winter
1950-01-02,22878701.0,36.0,-5.710409,-5.799340,NaN,NaN,NaN,NaN,NaN,NaN,...,0.380004,1950.0,2.0,-7304.0,29376.0,8.160,69.880874,216.938301,365.0,winter
1950-01-03,22878701.0,36.0,-4.825355,-4.825355,-4.647132,NaN,NaN,NaN,NaN,NaN,...,0.449883,1950.0,3.0,-7303.0,29376.0,8.160,69.880874,216.938301,365.0,winter
1950-01-04,22878701.0,36.0,-7.213134,-7.213134,-5.309051,NaN,NaN,NaN,NaN,NaN,...,0.459806,1950.0,4.0,-7302.0,29721.6,8.256,69.880874,216.938301,365.0,winter
1950-01-05,22878701.0,36.0,-8.055808,-8.055808,-7.449383,NaN,NaN,NaN,NaN,NaN,...,0.461339,1950.0,5.0,-7301.0,29721.6,8.256,69.880874,216.938301,365.0,winter
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2099-12-27,22878701.0,36.0,-0.654279,-1.229499,0.122887,-0.263343,0.480638,0.428873,0.732779,0.678861,...,1.126416,2099.0,361.0,47477.0,29030.4,8.064,12.746725,16.016532,365.0,winter
2099-12-28,22878701.0,36.0,-1.051458,-1.099073,-0.801696,-0.712857,0.186350,0.202987,0.462227,0.662230,...,1.012965,2099.0,362.0,47478.0,29030.4,8.064,12.746725,16.016532,365.0,winter
2099-12-29,22878701.0,36.0,-0.711673,-1.086708,-1.127412,-0.829358,-0.136890,0.282881,0.304646,0.587516,...,0.703352,2099.0,363.0,47479.0,29376.0,8.160,12.746725,16.016532,365.0,winter


In [213]:
for i in [range(len(temp_files_gcms))[0]]:
    cur_file = temp_files_gcms[i]
    cur_huc = int(cur_file.split('/')[-1][:-3])

    preds = xr.open_dataset(cur_file, decode_times=True)
    predsdf = preds.to_dataframe(dim_order = ['COMID', 'GCM', 'date'])
    preds_flat = predsdf.reset_index()

In [213]:
for i in [range(len(temp_files_gcms))[0]]:
    cur_file = temp_files_gcms[i]
    cur_huc = int(cur_file.split('/')[-1][:-3])

    preds = xr.open_dataset(cur_file, decode_times=True)
    predsdf = preds.to_dataframe(dim_order = ['COMID', 'GCM', 'date'])
    preds_flat = predsdf.reset_index()

In [214]:
preds_flat

,COMID,GCM,date,prd.stream_temp
0,22878641,1,1950-01-01,NaN
1,22878641,1,1950-01-02,NaN
2,22878641,1,1950-01-03,0.000000
3,22878641,1,1950-01-04,0.000000
4,22878641,1,1950-01-05,0.000000
...,...,...,...,...
16983965,22879621,10,2099-12-27,1.924742
16983966,22879621,10,2099-12-28,1.892086
16983967,22879621,10,2099-12-29,1.316455
16983968,22879621,10,2099-12-30,2.847338


In [ ]:
#if predsdf.shape[0] > 0:

In [167]:
    cur_huc_anoms = []
    cur_comids = list(df.COMID.unique())

In [178]:
    for j in [cur_comids[0]]:
        print(j)
        cur_preds = df[df.COMID == j]
        cur_ens_med = df.groupby('date').median().drop('GCM', axis =1)
        #cur_ens_med['doy'] = cur_ens_med.index.dayofyear
        cur_ens_med['season'] = cur_ens_med['tim.doy'].apply(assign_season)

22878621


TypeError: agg function failed [how->median,dtype->object]

In [87]:
%%time
cur_huc_anoms = []
for j in cur_comids:
    print(j)
    cur_preds = df[df.COMID == j][cols]
    cur_ens_med = cur_preds.drop('GCM', axis = 1).groupby('date').median()
    #cur_ens_med['doy'] = cur_ens_med.index.dayofyear
    cur_ens_med['season'] = cur_ens_med['tim.doy'].apply(assign_season)

    cur_anomolies = []
    for dec in periods_idx:
        if dec == '2020s': 
            cur = cur_ens_med[cur_ens_med.index.year.isin(periods[dec])].groupby('season').median()#['prd.stream_temp']
            cur_anomolies.append(cur)
        else: cur_anomolies.append(cur_ens_med[cur_ens_med.index.year.isin(periods[dec])].groupby('season').median().subtract(cur))
    cur_seas_anomolies = pd.concat(cur_anomolies, axis = 1, keys = periods_idx)
    cur_huc_anoms.append(cur_seas_anomolies)

huc_anoms = pd.concat(cur_huc_anoms, keys = cur_comids)
huc_anoms = huc_anoms.reset_index(names = ['COMID', 'season'])
huc_anoms.to_csv(os.path.join(temp_dir_gcms, 'seasonal_anomolies', f'{cur_huc}_cov_anoms.csv'))
with py7zr.SevenZipFile(os.path.join(temp_dir_gcms, 'seasonal_anomolies', f'{cur_huc}_cov_anoms.7z'), 'w') as archive:
    archive.write(os.path.join(temp_dir_gcms, 'seasonal_anomolies', f'{cur_huc}_cov_anoms.csv'))
os.remove(os.path.join(temp_dir_gcms, 'seasonal_anomolies', f'{cur_huc}_cov_anoms.csv')) 

22878641
22878643
22878645
22878647
22878649
22878651
22878653
22878655
22878657
22878743
22878745
22878747
22878749
22878751
22878753
22878755
22878757
22878759
22878761
22879621
22878665
22878667
22878699
22878701
22878703
22878705
22878707
22878709
22878659
22878661
22878663
CPU times: user 3.32 s, sys: 43.5 ms, total: 3.37 s
Wall time: 4.24 s


In [ ]:
gcm_frames = []
fail_list = []
for cur_gcm in gcms:
    try: gcm_frames.append(ro.r(f"read.fst('/Volumes/Elements/URycki/temp_data/covariates_GCM/{cur_gcm}/huc_{cur_huc}.fst')"))
    except: fail_list.append(cur_gcm)

In [31]:
hucs

['1701010107',
 '1701010108',
 '1701010109',
 '1701010110',
 '1701010111',
 '1701010112',
 '1701010113',
 '1701010114',
 '1701010115',
 '1701010116',
 '1701010117',
 '1701010201',
 '1701010202',
 '1701010203',
 '1701010204',
 '1701010301',
 '1701010302',
 '1701010303',
 '1701010304',
 '1701010305',
 '1701010401',
 '1701010402',
 '1701010403',
 '1701010404',
 '1701010405',
 '1701010406',
 '1701010407',
 '1701010504',
 '1701010505',
 '1701020101',
 '1701020102',
 '1701020103',
 '1701020104',
 '1701020105',
 '1701020106',
 '1701020107',
 '1701020108',
 '1701020201',
 '1701020202',
 '1701020203',
 '1701020204',
 '1701020205',
 '1701020206',
 '1701020207',
 '1701020208',
 '1701020209',
 '1701020210',
 '1701020211',
 '1701020212',
 '1701020213',
 '1701020214',
 '1701020301',
 '1701020302',
 '1701020303',
 '1701020304',
 '1701020305',
 '1701020306',
 '1701020307',
 '1701020308',
 '1701020309',
 '1701020310',
 '1701020311',
 '1701020312',
 '1701020313',
 '1701020401',
 '1701020402',
 '17010204

In [38]:
i = 303


cur_huc = hucs[i]
print(cur_huc)

1704010409


In [ ]:
%%time
fail_list = {}

for i in range(303, len(hucs)):
    cur_huc = hucs[i]
    print(cur_huc)
    
    gcm_frames = []
    for cur_gcm in gcms:
        try: gcm_frames.append(ro.r(f"read.fst('/Volumes/Elements/URycki/temp_data/covariates_GCM/{cur_gcm}/huc_{cur_huc}.fst')"))
        except: fail_list['cur_huc'] = cur_gcm  # update this to check if fail_list['cur_huc'] exists; append cur_gcm
    df = pd.concat(gcm_frames, keys = gcms, names = ['GCM'])
    df = df.reset_index().drop('level_1', axis = 1)
    df['date'] = pd.to_datetime(df['tim.year'] * 1000 + df['tim.doy'], format='%Y%j')
    print(df.shape)
    
    if (cur_huc not in fail_list.keys()) & (df.shape[0] > 0):
        cur_comids = list(df.COMID.unique())
        print(len(cur_comids), ' COMIDs...')
        cur_huc_anoms = [] 
        for j in cur_comids:
            cur_preds = df[df.COMID == j][cols]
            cur_ens_med = cur_preds.drop('GCM', axis = 1).groupby('date').median()
            cur_ens_med['season'] = cur_ens_med['tim.doy'].apply(assign_season)
        
            cur_anomolies = []
            for dec in periods_idx:
                if dec == '2020s': 
                    cur = cur_ens_med[cur_ens_med.index.year.isin(periods[dec])].groupby('season').median()#['prd.stream_temp']
                    cur_anomolies.append(cur)
                else: cur_anomolies.append(cur_ens_med[cur_ens_med.index.year.isin(periods[dec])].groupby('season').median().subtract(cur))
            cur_seas_anomolies = pd.concat(cur_anomolies, axis = 1, keys = periods_idx)
            cur_huc_anoms.append(cur_seas_anomolies)
        
        huc_anoms = pd.concat(cur_huc_anoms, keys = cur_comids)
        huc_anoms = huc_anoms.reset_index(names = ['COMID', 'season'])
        huc_anoms.to_csv(os.path.join(temp_dir_gcms, 'seasonal_anomolies', f'{cur_huc}_cov_anoms.csv'))
        with py7zr.SevenZipFile(os.path.join(temp_dir_gcms, 'seasonal_anomolies', f'{cur_huc}_cov_anoms.7z'), 'w') as archive:
            archive.write(os.path.join(temp_dir_gcms, 'seasonal_anomolies', f'{cur_huc}_cov_anoms.csv'))
        os.remove(os.path.join(temp_dir_gcms, 'seasonal_anomolies', f'{cur_huc}_cov_anoms.csv')) 
        print('Done')
    else: pass

R[write to console]: fstcore package v0.9.18

R[write to console]: (OpenMP detected, using 16 threads)



1704010405


<timed exec>:11: FutureWarning: The behavior of pd.concat with len(keys) != len(objs) is deprecated. In a future version this will raise instead of truncating to the smaller of the two sequences


       GCM     COMID  SegID       lookup  cov.air_temp  cov.air_temp_ws  \
0  CanESM2  24439372   5302  5302_1950_1     -3.219251        -3.625504   
1  CanESM2  24439374   5302  5302_1950_1     -3.219251        -3.625504   
2  CanESM2  24439376   5302  5302_1950_1     -3.219251        -3.625504   
3  CanESM2  24439378   5302  5302_1950_1     -3.219251        -3.625504   
4  CanESM2  24439380   5302  5302_1950_1     -3.219251        -3.625504   

   cov.air_temp_3d  cov.air_temp_6d  cov.air_temp_9d  cov.air_temp_12d  ...  \
0              NaN              NaN              NaN               NaN  ...   
1              NaN              NaN              NaN               NaN  ...   
2              NaN              NaN              NaN               NaN  ...   
3              NaN              NaN              NaN               NaN  ...   
4              NaN              NaN              NaN               NaN  ...   

   cov.Flow_log  tim.year  tim.doy  tim.date  cov.daylength  \
0     -0.16

In [ ]:
print(fail_list)

In [103]:
if (cur_huc not in fail_list.keys()) & (df.shape[0] > 0): print('ok')


ok


In [101]:
df.shape[0] > 0

True

In [ ]:
        cur_anomolies = {}
        for dec in list(periods.keys()):
            if dec == '2020s': cur_anomolies[dec] = cur_ens_med[cur_ens_med.index.year.isin(periods[dec])].groupby('season').median()['prd.stream_temp']
            else: cur_anomolies[dec] = cur_ens_med[cur_ens_med.index.year.isin(periods[dec])].groupby('season').median()['prd.stream_temp'].subtract(cur_anomolies['2020s'])

        cur_seas_anomolies = pd.DataFrame(cur_anomolies)
        cur_huc_anoms.append(cur_seas_anomolies)
    
    huc_anoms = pd.concat(cur_huc_anoms, keys = cur_comids)
    huc_anoms = huc_anoms.reset_index(names = ['COMID', 'season'])
    huc_anoms.to_csv(os.path.join(temp_dir_gcms, 'seasonal_anomolies', f'{cur_huc}_anoms.csv'))
    with py7zr.SevenZipFile(os.path.join(temp_dir_gcms, 'seasonal_anomolies', f'{cur_huc}_anoms.7z'), 'w') as archive:
        archive.write(os.path.join(temp_dir_gcms, 'seasonal_anomolies', f'{cur_huc}_anoms.csv'))
    os.remove(os.path.join(temp_dir_gcms, 'seasonal_anomolies', f'{cur_huc}_anoms.csv')) 

In [ ]:
# Actually, these should be .zip instead of .7z for Riverscapes:
import zipfile 

with py7zr.SevenZipFile(f'/Volumes/Elements/URycki/temp_data/preds_GCM/seasonal_anomolies/{cur_huc}_anoms.7z', mode='r') as archive:
    archive.extractall(path = '/.')
sevenz_csv = pd.read_csv(f'/Volumes/Elements/URycki/temp_data/preds_GCM/seasonal_anomolies/{cur_huc}_anoms.csv')
sevenz_csv.rename(columns={'Unnamed: 0': "COMID"}, inplace = True)

with zipfile.ZipFile(f'/Volumes/Elements/URycki/temp_data/preds_GCM/seasonal_anomolies/{cur_huc}_anoms.zip', mode='w') as zipf:
    zipf.write(csv_filename)

In [36]:
cur_spatial_data = spatial_data_fill.loc[cur_comids_preds]
len(cur_spatial_data)

42

In [37]:
cur_spatial_data.head()

,cov.length_km,cov.stream_order,cov.path_length,cov.tot.da.sqkm,cov.elev_max_smo,cov.elev_min_smo,cov.slope,cov.sinuosity,cov.lat_v,cov.elev_mean_smo,...,cov.canopy_rca,cov.canopy_line,cov.area_sqkm,cov.canopy_ws,cov.dam_name,cov.proportion_dam_influenced,cov.distance_below_dam,cov.dam_hgt_m,cov.dam_main_use,geometry
COMID,,,,,,,,,,,,,,,,,,,,,
23066771,2.724,6.0,968.491,9253.2636,27630.0,27621.0,0.000033,2.49,48.908100,276.255,...,69.0,60.0,1.7352,74.129544,ENLOE DAM,0.994167,11.094,16.0,Hydroelectricity,"LINESTRING Z (-119.43344 48.91756 0, -119.4336..."
23066773,2.166,6.0,972.198,9243.9198,27959.0,27724.0,0.001085,1.06,48.930301,278.415,...,68.0,64.5,1.9413,74.161480,ENLOE DAM,0.995172,7.387,16.0,Hydroelectricity,"LINESTRING Z (-119.44774 48.93961 0, -119.4457..."
23066775,0.317,6.0,974.364,9240.2217,28013.0,27959.0,0.001703,1.00,48.936500,279.860,...,71.5,69.0,3.0618,74.175709,ENLOE DAM,0.995571,5.221,16.0,Hydroelectricity,"LINESTRING Z (-119.4506 48.94175 0, -119.44774..."
23067095,0.524,6.0,971.674,9244.3338,27724.0,27655.0,0.001317,1.22,48.920101,276.895,...,64.5,60.0,1.3131,74.158989,ENLOE DAM,0.995128,7.911,16.0,Hydroelectricity,"LINESTRING Z (-119.44245 48.92137 0, -119.4421..."
23067097,0.459,6.0,971.215,9244.4967,27655.0,27630.0,0.000545,1.22,48.921600,276.425,...,60.0,60.0,1.1565,74.157552,ENLOE DAM,0.995110,8.370,16.0,Hydroelectricity,"LINESTRING Z (-119.43706 48.91995 0, -119.4365..."


In [28]:
%%time
gpd.GeoDataFrame(cur_spatial_data).to_file(cur_output, layer = 'spatial_covariates', driver = "GPKG", layer_metadata = {'Fill value' : '-9999'}, dataset_metadata = nc_metadata)

CPU times: user 28.4 ms, sys: 10.9 ms, total: 39.4 ms
Wall time: 53.1 ms


In [78]:
#pyogrio.list_layers(cur_output)

In [79]:
#pyogrio.list_layers(cur_output)

In [80]:
#pyogrio.read_info(cur_output)

In [81]:
#test_gdf = gpd.read_file(cur_output)

In [82]:
#test_gdf

In [34]:
#test_gdf.explore()

In [83]:
%%time
#gpd.GeoDataFrame(preds_flat).to_file(cur_output, layer = 'stream_temperature_predictions', driver = "GPKG", layer_metadata = {'Fill_value' : '-9999'})

CPU times: user 4 μs, sys: 2 μs, total: 6 μs
Wall time: 8.82 μs


In [84]:
#pyogrio.read_info(cur_output)

In [37]:
# Now add temporal covariates


In [38]:
# Now add spatial covariates metadata
gpd.GeoDataFrame(covariate_metadata).to_file(cur_output, layer = 'covariate_metadata', driver = "GPKG")

In [39]:
# Now add GCM metadata
gpd.GeoDataFrame(gcm_metadata).to_file(cur_output, layer = 'GCM_metadata', driver = "GPKG")

In [85]:
#pyogrio.list_layers(cur_output)

In [86]:
#pyogrio.read_info(cur_output, layer = 'GCM_metadata')

In [42]:
huc

1702000720

In [48]:
gcms

['CanESM2',
 'CCSM4',
 'CNRM-CM5',
 'CSIRO-Mk3-6-0',
 'GDFL-ESM2M',
 'HadGEM-CC365',
 'HadGEM2-ES',
 'inmcm4',
 'IPSL-CM5A-MR',
 'MIROC5']

In [49]:
cur_gcm = gcms[0]

In [92]:
%%time
for cur_gcm in [gcms[0]]: 
    try:
        print(cur_gcm)
        cur_df = ro.r(f"read.fst('DATA/{cur_gcm}/huc_{huc}.fst')")
        cur_df_fill = cur_df.fillna(-9999)
        #gpd.GeoDataFrame(cur_df_fill).to_file(cur_output, layer = f'temporal_covariates_{cur_gcm}', driver = "GPKG", layer_metadata = {'Fill value': '-9999'})
    except: pass

CanESM2
CPU times: user 51.4 s, sys: 6.78 s, total: 58.2 s
Wall time: 49.1 s


In [94]:
CanESM2_df = cur_df.copy()

In [103]:
CanESM2_df.head()

,COMID,SegID,lookup,cov.air_temp,cov.air_temp_ws,cov.air_temp_3d,cov.air_temp_6d,cov.air_temp_9d,cov.air_temp_12d,cov.air_temp_15d,...,cov.Flow,cov.Flow_log,tim.year,tim.doy,tim.date,cov.daylength,cov.daylength_hours,cov.SWE_mean_year,cov.SWE_1Apr,cov.SWE_last_doy
1,24120706,4126,4126_1950_1,-16.526405,-21.096784,NaN,NaN,NaN,NaN,NaN,...,0.212842,-1.547207,1950.0,1.0,-7305.0,29030.4,8.064,61.418016,139.427406,365.0
2,24120706,4126,4126_1950_10,-11.314290,-16.989374,-8.728696,-12.190027,-9.953042,NaN,NaN,...,0.398822,-0.919239,1950.0,10.0,-7296.0,30067.2,8.352,61.418016,139.427406,365.0
3,24120706,4126,4126_1950_100,4.793318,0.504388,4.170258,7.144744,5.998789,5.943844,5.511841,...,1.051988,0.050682,1950.0,100.0,-7206.0,47347.2,13.152,61.418016,139.427406,365.0
4,24120706,4126,4126_1950_101,5.475709,2.091127,4.732936,6.402701,6.336809,5.980180,5.563224,...,1.153476,0.142780,1950.0,101.0,-7205.0,47692.8,13.248,61.418016,139.427406,365.0
5,24120706,4126,4126_1950_102,7.709150,3.441271,5.992725,5.572593,6.789519,6.092021,5.865226,...,1.340546,0.293077,1950.0,102.0,-7204.0,47692.8,13.248,61.418016,139.427406,365.0


In [ ]:
gcm

In [102]:
CanESM2_df['tim.date']

1          -7305.0
2          -7296.0
3          -7206.0
4          -7205.0
5          -7204.0
            ...   
2629772    47214.0
2629773    47215.0
2629774    47215.0
2629775    47215.0
2629776    47215.0
Name: tim.date, Length: 2629776, dtype: float64

In [96]:
len_CanESM2 = len(CanESM2_df)

In [87]:
#pyogrio.list_layers(cur_output)

In [88]:
#pyogrio.read_info(cur_output, layer = 'temporal_covariates_CanESM2')

In [89]:
#cur_df_fill.head

In [90]:
%%time
#gpd.GeoDataFrame(df_fill).to_file(cur_output, layer = f'temporal_covariates_{cur_gcm}', driver = "GPKG", layer_metadata = {'Fill value': '-9999'})

CPU times: user 5 μs, sys: 2 μs, total: 7 μs
Wall time: 14.1 μs


In [52]:
cur_output

'DATA/gpkgs/1702000720.gpkg'

In [55]:
df_all_gcm

,lookup,COMID,cov.air_temp,cov.air_temp_ws,cov.air_temp_3d,cov.air_temp_6d,cov.air_temp_9d,cov.air_temp_12d,cov.air_temp_15d,cov.air_temp_18d,...,tim.date,cov.SWE,cov.SWE_ws,cov.NWM_flow,cov.NWM_flow_log,cov.daylength,cov.daylength_hours,cov.SWE_mean_year,cov.SWE_1Apr,cov.SWE_last_doy
1,23065579_1990_1,23065579,-1.991000,-1.991000,NaN,NaN,NaN,NaN,NaN,NaN,...,7306.0,17.0,17.0,0.00001,-11.512925,29030.4,8.064,9.123288,0.0,169.0
2,23065579_1990_10,23065579,1.026000,1.026000,2.291000,0.945667,-0.786667,NaN,NaN,NaN,...,7315.0,25.0,25.0,0.00001,-11.512925,30067.2,8.352,9.123288,0.0,169.0
3,23065579_1990_100,23065579,6.658000,6.658000,9.664000,10.267834,10.465778,10.382167,9.722800,8.274000,...,7405.0,0.0,0.0,0.01000,-4.605170,47347.2,13.152,9.123288,0.0,169.0
4,23065579_1990_101,23065579,7.245000,7.245000,7.912667,9.778667,9.962223,10.247250,9.800200,8.697945,...,7406.0,0.0,0.0,0.01000,-4.605170,47692.8,13.248,9.123288,0.0,169.0
5,23065579_1990_102,23065579,8.252001,8.252001,7.385001,9.352167,9.568445,10.108500,9.824334,9.100612,...,7407.0,0.0,0.0,0.01000,-4.605170,47692.8,13.248,9.123288,0.0,169.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
879865,24961737_2022_95,24961737,1.504800,1.504800,3.237633,3.112833,4.702578,4.772358,4.446227,3.858122,...,19088.0,NaN,NaN,0.00001,-11.512925,46310.4,12.864,NaN,NaN,NaN
879866,24961737_2022_96,24961737,2.046400,2.046400,2.468167,2.798600,3.893344,4.730658,4.537620,3.822678,...,19089.0,NaN,NaN,0.00001,-11.512925,46656.0,12.960,NaN,NaN,NaN
879867,24961737_2022_97,24961737,2.149300,2.149300,1.900167,2.755983,3.290733,4.446558,4.417300,3.920683,...,19090.0,NaN,NaN,0.00001,-11.512925,46656.0,12.960,NaN,NaN,NaN
879868,24961737_2022_98,24961737,6.794600,6.794600,3.663433,3.450533,3.296367,4.442792,4.550573,4.315761,...,19091.0,NaN,NaN,0.00001,-11.512925,47001.6,13.056,NaN,NaN,NaN


In [104]:
df_all_gcm.columns

Index(['lookup', 'COMID', 'cov.air_temp', 'cov.air_temp_ws', 'cov.air_temp_3d',
       'cov.air_temp_6d', 'cov.air_temp_9d', 'cov.air_temp_12d',
       'cov.air_temp_15d', 'cov.air_temp_18d', 'cov.air_temp_21d',
       'cov.air_temp_24d', 'cov.air_temp_27d', 'cov.air_temp_30d',
       'cov.air_temp_35d', 'cov.air_temp_40d', 'cov.air_temp_50d',
       'cov.air_temp_60d', 'Huc10', 'tim.year', 'tim.doy', 'tim.date',
       'cov.SWE', 'cov.SWE_ws', 'cov.NWM_flow', 'cov.NWM_flow_log',
       'cov.daylength', 'cov.daylength_hours', 'cov.SWE_mean_year',
       'cov.SWE_1Apr', 'cov.SWE_last_doy'],
      dtype='object')

In [66]:
df_fill

,lookup,COMID,cov.air_temp,cov.air_temp_ws,cov.air_temp_3d,cov.air_temp_6d,cov.air_temp_9d,cov.air_temp_12d,cov.air_temp_15d,cov.air_temp_18d,...,tim.date,cov.SWE,cov.SWE_ws,cov.NWM_flow,cov.NWM_flow_log,cov.daylength,cov.daylength_hours,cov.SWE_mean_year,cov.SWE_1Apr,cov.SWE_last_doy
1,23065579_1990_1,23065579,-1.991000,-1.991000,NaN,NaN,NaN,NaN,NaN,NaN,...,7306.0,17.0,17.0,0.00001,-11.512925,29030.4,8.064,9.123288,0.0,169.0
2,23065579_1990_10,23065579,1.026000,1.026000,2.291000,0.945667,-0.786667,NaN,NaN,NaN,...,7315.0,25.0,25.0,0.00001,-11.512925,30067.2,8.352,9.123288,0.0,169.0
3,23065579_1990_100,23065579,6.658000,6.658000,9.664000,10.267834,10.465778,10.382167,9.722800,8.274000,...,7405.0,0.0,0.0,0.01000,-4.605170,47347.2,13.152,9.123288,0.0,169.0
4,23065579_1990_101,23065579,7.245000,7.245000,7.912667,9.778667,9.962223,10.247250,9.800200,8.697945,...,7406.0,0.0,0.0,0.01000,-4.605170,47692.8,13.248,9.123288,0.0,169.0
5,23065579_1990_102,23065579,8.252001,8.252001,7.385001,9.352167,9.568445,10.108500,9.824334,9.100612,...,7407.0,0.0,0.0,0.01000,-4.605170,47692.8,13.248,9.123288,0.0,169.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
879865,24961737_2022_95,24961737,1.504800,1.504800,3.237633,3.112833,4.702578,4.772358,4.446227,3.858122,...,19088.0,NaN,NaN,0.00001,-11.512925,46310.4,12.864,NaN,NaN,NaN
879866,24961737_2022_96,24961737,2.046400,2.046400,2.468167,2.798600,3.893344,4.730658,4.537620,3.822678,...,19089.0,NaN,NaN,0.00001,-11.512925,46656.0,12.960,NaN,NaN,NaN
879867,24961737_2022_97,24961737,2.149300,2.149300,1.900167,2.755983,3.290733,4.446558,4.417300,3.920683,...,19090.0,NaN,NaN,0.00001,-11.512925,46656.0,12.960,NaN,NaN,NaN
879868,24961737_2022_98,24961737,6.794600,6.794600,3.663433,3.450533,3.296367,4.442792,4.550573,4.315761,...,19091.0,NaN,NaN,0.00001,-11.512925,47001.6,13.056,NaN,NaN,NaN


In [69]:
df_fill_set = set(df_fill.COMID.unique())

In [70]:
df_fill_set

{np.int64(23066771),
 np.int64(23066773),
 np.int64(23066775),
 np.int64(23067095),
 np.int64(23067097),
 np.int64(23067181),
 np.int64(24120666),
 np.int64(24120668),
 np.int64(24120670),
 np.int64(24120672),
 np.int64(24120674),
 np.int64(24120676),
 np.int64(24120678),
 np.int64(24120680),
 np.int64(24120682),
 np.int64(24120684),
 np.int64(24120686),
 np.int64(24120688),
 np.int64(24120690),
 np.int64(24120692),
 np.int64(24120694),
 np.int64(24120696),
 np.int64(24120698),
 np.int64(24120700),
 np.int64(24120704),
 np.int64(24120706),
 np.int64(24120708),
 np.int64(24120710),
 np.int64(24120712),
 np.int64(24120714),
 np.int64(24120716),
 np.int64(24120718),
 np.int64(24120738),
 np.int64(24121446),
 np.int64(24121550),
 np.int64(24121932),
 np.int64(24121936),
 np.int64(24121938),
 np.int64(24121946),
 np.int64(24121948),
 np.int64(24121950),
 np.int64(24121952),
 np.int64(24121954),
 np.int64(24121956),
 np.int64(24121962),
 np.int64(24121964),
 np.int64(24121984),
 np.int64(241

In [71]:
df_set = set(df.COMID.unique())

In [72]:
df_set

{np.int64(23065579),
 np.int64(23066771),
 np.int64(23066773),
 np.int64(23066775),
 np.int64(23066855),
 np.int64(23067095),
 np.int64(23067097),
 np.int64(23067177),
 np.int64(23067179),
 np.int64(23067181),
 np.int64(24120666),
 np.int64(24120668),
 np.int64(24120670),
 np.int64(24120672),
 np.int64(24120674),
 np.int64(24120676),
 np.int64(24120678),
 np.int64(24120680),
 np.int64(24120682),
 np.int64(24120684),
 np.int64(24120686),
 np.int64(24120688),
 np.int64(24120690),
 np.int64(24120692),
 np.int64(24120694),
 np.int64(24120696),
 np.int64(24120698),
 np.int64(24120700),
 np.int64(24120704),
 np.int64(24120706),
 np.int64(24120708),
 np.int64(24120710),
 np.int64(24120712),
 np.int64(24120714),
 np.int64(24120716),
 np.int64(24121480),
 np.int64(24121484),
 np.int64(24121492),
 np.int64(24121510),
 np.int64(24121528),
 np.int64(24121540),
 np.int64(24121542),
 np.int64(24121544),
 np.int64(24121548),
 np.int64(24121550),
 np.int64(24121552),
 np.int64(24121570),
 np.int64(241

In [74]:
df_fill_set.difference(df_set)

{np.int64(24120718),
 np.int64(24120738),
 np.int64(24121446),
 np.int64(24121946),
 np.int64(24121948),
 np.int64(24121956),
 np.int64(24121962),
 np.int64(24121990)}

In [76]:
spatial_data_all


,COMID,cov.length_km,cov.stream_order,cov.path_length,cov.tot.da.sqkm,cov.elev_max_smo,cov.elev_min_smo,cov.slope,cov.sinuosity,cov.lat_v,...,cov.pct_urb_all_ws,cov.canopy_rca,cov.canopy_line,cov.area_sqkm,cov.canopy_ws,cov.dam_name,cov.proportion_dam_influenced,cov.distance_below_dam,cov.dam_hgt_m,cov.dam_main_use
0,9301535,0.999,NaN,NaN,NaN,NaN,NaN,NaN,1.47,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,22226592,0.733,NaN,NaN,NaN,NaN,NaN,NaN,1.13,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,22226600,6.117,NaN,NaN,NaN,NaN,NaN,NaN,1.62,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,22877591,0.247,6.0,1290.195,28030.5369,56102.0,55932.0,0.006883,1.00,48.558399,...,0.946698,89.000000,90.0,1.9161,88.592748,Libby,0.831332,71.294,129.0,Hydroelectricity
4,22877593,0.055,3.0,1290.442,58.4136,56129.0,56102.0,0.004909,1.00,48.556999,...,0.000000,90.000000,90.0,0.4284,90.227047,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270936,947120097,0.161,NaN,NaN,NaN,NaN,NaN,NaN,1.00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
270937,947120098,1.218,2.0,40.722,10.9971,145858.0,139672.0,0.050788,1.04,42.388000,...,0.000000,82.333333,82.0,0.4815,82.684671,NaN,NaN,NaN,NaN,NaN
270938,947120099,0.366,NaN,NaN,NaN,NaN,NaN,NaN,1.01,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
270939,947120100,2.893,3.0,39.940,44.0622,140135.0,138629.0,0.005206,1.12,42.368198,...,0.449365,81.000000,78.0,0.2196,80.865589,NaN,NaN,NaN,NaN,NaN


In [77]:
cur_spatial_data

,cov.length_km,cov.stream_order,cov.path_length,cov.tot.da.sqkm,cov.elev_max_smo,cov.elev_min_smo,cov.slope,cov.sinuosity,cov.lat_v,cov.elev_mean_smo,...,cov.canopy_rca,cov.canopy_line,cov.area_sqkm,cov.canopy_ws,cov.dam_name,cov.proportion_dam_influenced,cov.distance_below_dam,cov.dam_hgt_m,cov.dam_main_use,geometry
COMID,,,,,,,,,,,,,,,,,,,,,
23066771,2.724,6.0,968.491,9253.2636,27630.0,27621.0,0.000033,2.49,48.908100,276.255,...,69.000000,60.000000,1.7352,74.129544,ENLOE DAM,0.994167,11.094,16.0,Hydroelectricity,"LINESTRING Z (-119.43344 48.91756 0, -119.4336..."
23066773,2.166,6.0,972.198,9243.9198,27959.0,27724.0,0.001085,1.06,48.930301,278.415,...,68.000000,64.500000,1.9413,74.161480,ENLOE DAM,0.995172,7.387,16.0,Hydroelectricity,"LINESTRING Z (-119.44774 48.93961 0, -119.4457..."
23066775,0.317,6.0,974.364,9240.2217,28013.0,27959.0,0.001703,1.00,48.936500,279.860,...,71.500000,69.000000,3.0618,74.175709,ENLOE DAM,0.995571,5.221,16.0,Hydroelectricity,"LINESTRING Z (-119.4506 48.94175 0, -119.44774..."
23067095,0.524,6.0,971.674,9244.3338,27724.0,27655.0,0.001317,1.22,48.920101,276.895,...,64.500000,60.000000,1.3131,74.158989,ENLOE DAM,0.995128,7.911,16.0,Hydroelectricity,"LINESTRING Z (-119.44245 48.92137 0, -119.4421..."
23067097,0.459,6.0,971.215,9244.4967,27655.0,27630.0,0.000545,1.22,48.921600,276.425,...,60.000000,60.000000,1.1565,74.157552,ENLOE DAM,0.995110,8.370,16.0,Hydroelectricity,"LINESTRING Z (-119.43706 48.91995 0, -119.4365..."
23067181,1.941,6.0,965.317,9256.0887,27620.0,27620.0,0.000010,2.81,48.895198,276.200,...,68.333333,63.000000,1.0926,74.117784,ENLOE DAM,0.993864,14.268,16.0,Hydroelectricity,"LINESTRING Z (-119.43337 48.89747 0, -119.4335..."
24120666,3.146,6.0,974.681,9239.3883,28964.0,28013.0,0.003023,1.09,48.951099,284.885,...,74.000000,69.000000,7.1703,74.177102,ENLOE DAM,0.995660,4.904,16.0,Hydroelectricity,"LINESTRING Z (-119.48537 48.95368 0, -119.4848..."
24120668,0.742,6.0,977.827,9227.2221,29291.0,28964.0,0.004407,1.03,48.954200,291.275,...,71.500000,64.000000,2.0295,74.181981,ENLOE DAM,0.996973,1.758,16.0,Hydroelectricity,"LINESTRING Z (-119.49154 48.95875 0, -119.4884..."
24120670,0.757,6.0,978.569,9224.4483,30194.0,29291.0,0.011929,1.01,48.959201,297.425,...,67.500000,64.000000,1.4202,74.194093,ENLOE DAM,0.997273,1.016,16.0,Hydroelectricity,"LINESTRING Z (-119.4996 48.96282 0, -119.49797..."


In [78]:
len(cur_spatial_data)

42

In [79]:
len(df_fill_set)

48

In [80]:
len(df_set)

73

In [50]:
huc

1702000720

In [ ]:
gcm1 = 

In [240]:
gcm = 'GDFL-ESM2M'

In [31]:
def get_geometries(comids, hydrofile):
    flowlines_all = gpd.read_file(hydrofile)
    geometries = flowlines_all[['COMID', 'geometry']].set_index('COMID')
    return geometries

In [ ]:
def add_attribute_table(df, gpkg, layer_name = 'stream_temperature_predictions'):
    df["geometry"] = [None for i in df.index]
    attonly_schema = {"geometry": "None", "properties": df.dtypes.to_dict()}
    df.to_file(gpkg, layer = layer_name, driver = "GPKG", schema = attonly_schema)
    return None

In [49]:
df = gpd.GeoDataFrame(preds_flat)

In [50]:
df["geometry"] = [None for i in df.index]

In [51]:
attonly_schema = {"geometry": "None", "properties": df.dtypes.to_dict()}

In [52]:
gpkg = 'test_output.gpkg'

In [54]:
layer_name = 'stream_temperature_predictions'

In [58]:
%%time
gpd.GeoDataFrame(preds_flat).to_file(gpkg, layer = layer_name, driver = "GPKG")

/opt/miniconda3/envs/temp-data-env/lib/python3.12/site-packages/pyogrio/geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


CPU times: user 2min 8s, sys: 4.06 s, total: 2min 12s
Wall time: 2min 12s


In [59]:
gpd.read_file(cur)

/opt/miniconda3/envs/temp-data-env/lib/python3.12/site-packages/pyogrio/geopandas.py:265: UserWarning: More than one layer found in 'test_output.gpkg': 'test_output' (default), 'stream_temperature_predictions'. Specify layer parameter to avoid this warning.
  result = read_func(


,COMID,geometry
0,23066771,"LINESTRING Z (-119.43344 48.91756 0, -119.4336..."
1,23066773,"LINESTRING Z (-119.44774 48.93961 0, -119.4457..."
2,23066775,"LINESTRING Z (-119.4506 48.94175 0, -119.44774..."
3,23067095,"LINESTRING Z (-119.44245 48.92137 0, -119.4421..."
4,23067097,"LINESTRING Z (-119.43706 48.91995 0, -119.4365..."
5,23067181,"LINESTRING Z (-119.43337 48.89747 0, -119.4335..."
6,24120666,"LINESTRING Z (-119.48537 48.95368 0, -119.4848..."
7,24120668,"LINESTRING Z (-119.49154 48.95875 0, -119.4884..."
8,24120670,"LINESTRING Z (-119.4996 48.96282 0, -119.49797..."
9,24120672,"LINESTRING Z (-119.50067 48.96447 0, -119.5005..."


In [60]:
gpd.read_file(gpkg, layer = 'stream_temperature_predictions')

,COMID,GCM,date,prd.stream_temp,geometry
0,23066771,CanESM2,1950-01-01,NaN,None
1,23066771,CanESM2,1950-01-02,NaN,None
2,23066771,CanESM2,1950-01-03,NaN,None
3,23066771,CanESM2,1950-01-04,NaN,None
4,23066771,CanESM2,1950-01-05,NaN,None
...,...,...,...,...,...
23010535,24121984,MIROC5,2099-12-27,9.303043,None
23010536,24121984,MIROC5,2099-12-28,8.525719,None
23010537,24121984,MIROC5,2099-12-29,7.030612,None
23010538,24121984,MIROC5,2099-12-30,8.553759,None


In [61]:
gpd.list_layers(gpkg)

,name,geometry_type
0,test_output,LineString Z
1,stream_temperature_predictions,Unknown


In [65]:
import pyogrio


In [66]:
pyogrio.list_layers(gpkg)

array([['test_output', 'LineString Z'],
       ['stream_temperature_predictions', 'Unknown']], dtype=object)

In [67]:
pyogrio.read_info(gpkg)

/var/folders/95/99dy_ns968195cnlvp5xhwqm0000gp/T/ipykernel_15013/214239797.py:1: UserWarning: More than one layer found in 'test_output.gpkg': 'test_output' (default), 'stream_temperature_predictions'. Specify layer parameter to avoid this warning.
  pyogrio.read_info(gpkg)


{'layer_name': 'test_output',
 'crs': 'EPSG:4269',
 'encoding': 'UTF-8',
 'fields': array(['COMID'], dtype=object),
 'dtypes': array(['int32'], dtype=object),
 'fid_column': 'fid',
 'geometry_name': 'geom',
 'geometry_type': 'LineString Z',
 'features': 42,
 'total_bounds': (-119.709822791293,
  48.8388943241923,
  -119.429657325061,
  48.9890991906258),
 'driver': 'GPKG',
 'capabilities': {'random_read': True,
  'fast_set_next_by_index': True,
  'fast_spatial_filter': True,
  'fast_feature_count': True,
  'fast_total_bounds': True},
 'layer_metadata': None,
 'dataset_metadata': None}

In [32]:
# Load nc dataset 

def nc_to_gpkg(nc_filename, shortname_dict, dims = ['COMID', 'GCM', 'date']):
    preds = xr.open_dataset(cur_ncfile)
    predsdf = preds.to_dataframe(dim_order = dims)
    preds_flat = predsdf.reset_index()
    preds_flat.GCM = preds_flat.GCM.map(shortname_dict)

    # Retrieve geometries
    cur_comids = predsdf.index.get_level_values('COMID').unique()
    geoms = get_geometries(cur_comids, dbfhydro[0])

    # Create geopackage
    cur_geom.to_file('test_output.gpkg', driver = "GPKG")
    
    
    

In [42]:
preds_flat.dtypes.to_dict()

{'COMID': dtype('int32'),
 'GCM': dtype('O'),
 'date': dtype('<M8[ns]'),
 'prd.stream_temp': dtype('float64')}

In [17]:
gcm_metadata = 

In [18]:
gcms = gcm_metadata['Global Climate Model short name'].to_list()

In [19]:
gcms

['CanESM2',
 'CCSM4',
 'CNRM-CM5',
 'CSIRO-Mk3-6-0',
 'GDFL-ESM2M',
 'HadGEM-CC365',
 'HadGEM2-ES',
 'inmcm4',
 'IPSL-CM5A-MR',
 'MIROC5']

In [20]:
gcm_dict = {i+1:gcms[i] for i in list(range(10))}
gcm_dict

{1: 'CanESM2',
 2: 'CCSM4',
 3: 'CNRM-CM5',
 4: 'CSIRO-Mk3-6-0',
 5: 'GDFL-ESM2M',
 6: 'HadGEM-CC365',
 7: 'HadGEM2-ES',
 8: 'inmcm4',
 9: 'IPSL-CM5A-MR',
 10: 'MIROC5'}

In [15]:
150*365


54750

In [21]:
#conda install fstlib

In [22]:
#pip install fstlib


In [23]:
#from fstlib import fstlib

In [24]:
predsdf

prd.stream_temp
COMID    GCM date                       
23066771 1   1950-01-01              NaN
             1950-01-02              NaN
             1950-01-03              NaN
             1950-01-04              NaN
             1950-01-05              NaN
...                                  ...
24121984 10  2099-12-27         9.303043
             2099-12-28         8.525719
             2099-12-29         7.030612
             2099-12-30         8.553759
             2099-12-31         7.153554

[23010540 rows x 1 columns]

In [ ]:
predsdf

In [26]:
cur_comids = predsdf.index.get_level_values('COMID').unique()

In [27]:
len(predsdf.index.get_level_values('COMID').unique())

42

In [28]:
cur_comids

Index([23066771, 23066773, 23066775, 23067095, 23067097, 23067181, 24120666,
       24120668, 24120670, 24120672, 24120674, 24120676, 24120678, 24120680,
       24120682, 24120684, 24120686, 24120688, 24120690, 24120692, 24120694,
       24120696, 24120698, 24120700, 24120704, 24120706, 24120708, 24120710,
       24120712, 24120714, 24120716, 24120718, 24120738, 24121550, 24121932,
       24121936, 24121938, 24121950, 24121952, 24121954, 24121964, 24121984],
      dtype='int32', name='COMID')

In [18]:
#gcm1 = predsdf.xs(1, level = 1)

In [19]:
#gcm1

prd.stream_temp
COMID    date                       
23066771 1950-01-01              NaN
         1950-01-02              NaN
         1950-01-03              NaN
         1950-01-04              NaN
         1950-01-05              NaN
...                              ...
24121984 2099-12-27         5.770917
         2099-12-28         6.822931
         2099-12-29         7.623787
         2099-12-30         8.221618
         2099-12-31         7.753777

[2301054 rows x 1 columns]

In [41]:
import pandas as pd

In [64]:
geometries = flowlines_all[['COMID', 'geometry']].set_index('COMID')
geometries


,geometry
COMID,
9301535,"LINESTRING Z (-114.05855 48.99999 0, -114.0583..."
22968128,"LINESTRING Z (-113.00089 47.46274 0, -113.0020..."
22968138,"LINESTRING Z (-113.0013 47.45837 0, -113.00137..."
23135995,"LINESTRING Z (-111.06214 42.50841 0, -111.0619..."
23135997,"LINESTRING Z (-111.04516 42.51372 0, -111.0447..."
...,...
24545039,"LINESTRING Z (-118.12671 42.98327 0, -118.1260..."
24545041,"LINESTRING Z (-118.1111 42.99525 0, -118.11175..."
23706720,"LINESTRING Z (-121.60348 43.32724 0, -121.6034..."


In [43]:
#gcm1.index.get_level_values(0)

Index([23066771, 23066771, 23066771, 23066771, 23066771, 23066771, 23066771,
       23066771, 23066771, 23066771,
       ...
       24121984, 24121984, 24121984, 24121984, 24121984, 24121984, 24121984,
       24121984, 24121984, 24121984],
      dtype='int32', name='COMID', length=2301054)

In [21]:
#gcm1_flat = gcm1.reset_index()

In [22]:
#gcm1_flat


,COMID,date,prd.stream_temp
0,23066771,1950-01-01,NaN
1,23066771,1950-01-02,NaN
2,23066771,1950-01-03,NaN
3,23066771,1950-01-04,NaN
4,23066771,1950-01-05,NaN
...,...,...,...
2301049,24121984,2099-12-27,5.770917
2301050,24121984,2099-12-28,6.822931
2301051,24121984,2099-12-29,7.623787
2301052,24121984,2099-12-30,8.221618


In [66]:
preds_flat = predsdf.reset_index()
preds_flat

,COMID,GCM,date,prd.stream_temp
0,23066771,1,1950-01-01,NaN
1,23066771,1,1950-01-02,NaN
2,23066771,1,1950-01-03,NaN
3,23066771,1,1950-01-04,NaN
4,23066771,1,1950-01-05,NaN
...,...,...,...,...
23010535,24121984,10,2099-12-27,9.303043
23010536,24121984,10,2099-12-28,8.525719
23010537,24121984,10,2099-12-29,7.030612
23010538,24121984,10,2099-12-30,8.553759


In [73]:
cur_comids = preds_flat.COMID.unique()
cur_comids

array([23066771, 23066773, 23066775, 23067095, 23067097, 23067181,
       24120666, 24120668, 24120670, 24120672, 24120674, 24120676,
       24120678, 24120680, 24120682, 24120684, 24120686, 24120688,
       24120690, 24120692, 24120694, 24120696, 24120698, 24120700,
       24120704, 24120706, 24120708, 24120710, 24120712, 24120714,
       24120716, 24120718, 24120738, 24121550, 24121932, 24121936,
       24121938, 24121950, 24121952, 24121954, 24121964, 24121984],
      dtype=int32)

In [74]:
len(cur_comids)

42

In [70]:
geometries.index.unique()

Index([ 9301535, 22968128, 22968138, 23135995, 23135997, 23135999, 23136001,
       23136003, 23136005, 23136007,
       ...
       24545029, 24545031, 24545033, 24545035, 24545037, 24545039, 24545041,
       23706720, 23931942, 24279568],
      dtype='int32', name='COMID', length=272340)

In [71]:
#%%time
#gcm1_flat['geometries'] = gcm1_flat.COMID
#gcm1_flat

In [49]:
#geomdict = geometries.to_dict()

In [77]:
cur_geom = geometries.loc[cur_comids]
cur_geom

,geometry
COMID,
23066771,"LINESTRING Z (-119.43344 48.91756 0, -119.4336..."
23066773,"LINESTRING Z (-119.44774 48.93961 0, -119.4457..."
23066775,"LINESTRING Z (-119.4506 48.94175 0, -119.44774..."
23067095,"LINESTRING Z (-119.44245 48.92137 0, -119.4421..."
23067097,"LINESTRING Z (-119.43706 48.91995 0, -119.4365..."
23067181,"LINESTRING Z (-119.43337 48.89747 0, -119.4335..."
24120666,"LINESTRING Z (-119.48537 48.95368 0, -119.4848..."
24120668,"LINESTRING Z (-119.49154 48.95875 0, -119.4884..."
24120670,"LINESTRING Z (-119.4996 48.96282 0, -119.49797..."


In [78]:
len(cur_geom)

42

In [79]:
#for cid in gcm1_flat.geometries.unique():
#    print(geomdict[str(cid)])

In [51]:
#set2 = set(gcm1_flat.COMID.unique())

In [52]:
#gcm1_flat.COMID.unique().dtype

dtype('int32')

In [53]:
#set1 = set(geometries.index.unique())

In [54]:
#len(set2)

42

In [55]:
#len(set1.intersection(set2))

42

In [56]:
#geometries.index.unique().dtype

dtype('int32')

In [57]:
#a = gcm1_flat.geometries[0]

In [58]:
#geomdict[str(a)]

KeyError: '23066771'

In [59]:
list(geomdict.keys())

['geometry']

In [60]:
geometries

,geometry
COMID,
9301535,"LINESTRING Z (-114.05855 48.99999 0, -114.0583..."
22968128,"LINESTRING Z (-113.00089 47.46274 0, -113.0020..."
22968138,"LINESTRING Z (-113.0013 47.45837 0, -113.00137..."
23135995,"LINESTRING Z (-111.06214 42.50841 0, -111.0619..."
23135997,"LINESTRING Z (-111.04516 42.51372 0, -111.0447..."
...,...
24545039,"LINESTRING Z (-118.12671 42.98327 0, -118.1260..."
24545041,"LINESTRING Z (-118.1111 42.99525 0, -118.11175..."
23706720,"LINESTRING Z (-121.60348 43.32724 0, -121.6034..."


In [100]:
geometries.columns.values

array(['geometry'], dtype=object)

In [101]:
geometries.loc[a].values

<GeometryArray>
[<LINESTRING Z (-117.492 47.781 0, -117.492 47.781 0, -117.492 47.78 0, -117....>]
Length: 1, dtype: geometry

In [107]:
geometries.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 272340 entries, 9301535 to 24279568
Data columns (total 1 columns):
 #   Column    Non-Null Count   Dtype   
---  ------    --------------   -----   
 0   geometry  272340 non-null  geometry
dtypes: geometry(1)
memory usage: 9.2 MB


In [80]:
#gcm1_flat['COMID'].map(geometries)

In [113]:
geo_gcm1 = gpd.GeoDataFrame(gcm1_flat)

In [119]:
#trim_geos = geometries.loc[list(set2.intersection(set1))]

In [128]:
#trim_geos

,geometry
COMID,
23026816,"LINESTRING Z (-117.15635 47.96456 0, -117.1584..."
23026818,"LINESTRING Z (-117.14009 47.93361 0, -117.1400..."
23026820,"LINESTRING Z (-117.33586 47.92632 0, -117.3361..."
23026822,"LINESTRING Z (-117.33379 47.95037 0, -117.3337..."
23026824,"LINESTRING Z (-117.33587 47.95635 0, -117.3347..."
23026826,"LINESTRING Z (-117.33448 47.96387 0, -117.3348..."
23026828,"LINESTRING Z (-117.32955 47.98265 0, -117.3306..."
23026708,"LINESTRING Z (-117.49237 47.78126 0, -117.4922..."
23026710,"LINESTRING Z (-117.48148 47.77784 0, -117.4820..."


In [81]:
cur_geom.to_file('test_output.gpkg', driver = "GPKG")

In [82]:
pwd

'/Users/dawn.urycki/Repos/temp-data'

In [83]:
newdata = gpd.read_file('test_output.gpkg')

In [84]:
newdata

,COMID,geometry
0,23066771,"LINESTRING Z (-119.43344 48.91756 0, -119.4336..."
1,23066773,"LINESTRING Z (-119.44774 48.93961 0, -119.4457..."
2,23066775,"LINESTRING Z (-119.4506 48.94175 0, -119.44774..."
3,23067095,"LINESTRING Z (-119.44245 48.92137 0, -119.4421..."
4,23067097,"LINESTRING Z (-119.43706 48.91995 0, -119.4365..."
5,23067181,"LINESTRING Z (-119.43337 48.89747 0, -119.4335..."
6,24120666,"LINESTRING Z (-119.48537 48.95368 0, -119.4848..."
7,24120668,"LINESTRING Z (-119.49154 48.95875 0, -119.4884..."
8,24120670,"LINESTRING Z (-119.4996 48.96282 0, -119.49797..."
9,24120672,"LINESTRING Z (-119.50067 48.96447 0, -119.5005..."


In [85]:
type(newdata)

geopandas.geodataframe.GeoDataFrame

In [86]:
newdata.crs

<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - onshore and offshore: Canada - Alberta; British Columbia; Manitoba; New Brunswick; Newfoundland and Labrador; Northwest Territories; Nova Scotia; Nunavut; Ontario; Prince Edward Island; Quebec; Saskatchewan; Yukon. Puerto Rico. United States (USA) - Alabama; Alaska; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Hawaii; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming. US Virgin Islands. British Virgin Islands

In [92]:
newdata.to_file('output2.gpkg', layer = 'flowlines', driver = 'GPKG')

In [93]:
gpd.list_layers('output3.gpkg')

,name,geometry_type
0,flowlines,LineString Z


In [145]:
#gcm1_test = gcm1_flat.copy()

In [36]:
preds_test = preds_flat.copy()
preds_test

,COMID,GCM,date,prd.stream_temp
0,23066771,CanESM2,1950-01-01,NaN
1,23066771,CanESM2,1950-01-02,NaN
2,23066771,CanESM2,1950-01-03,NaN
3,23066771,CanESM2,1950-01-04,NaN
4,23066771,CanESM2,1950-01-05,NaN
...,...,...,...,...
23010535,24121984,MIROC5,2099-12-27,9.303043
23010536,24121984,MIROC5,2099-12-28,8.525719
23010537,24121984,MIROC5,2099-12-29,7.030612
23010538,24121984,MIROC5,2099-12-30,8.553759


In [98]:
gcm_dict

{1: 'CanESM2',
 2: 'CCSM4',
 3: 'CNRM-CM5',
 4: 'CSIRO-Mk3-6-0',
 5: 'GDFL-ESM2M',
 6: 'HadGEM-CC365',
 7: 'HadGEM2-ES',
 8: 'inmcm4',
 9: 'IPSL-CM5A-MR',
 10: 'MIROC5'}

In [37]:
preds_test.to_file('test_output.gpkg', layer = 'temp_predictions', driver = 'GPKG')

AttributeError: 'DataFrame' object has no attribute 'to_file'

In [96]:
gpd.list_layers('output3.gpkg')

,name,geometry_type
0,flowlines,LineString Z
1,temp_predictions,LineString Z


In [197]:
#geomdict

In [176]:
geometries.columns


Index(['geometry'], dtype='object')

In [177]:
gcm1_flat.columns

Index(['COMID', 'date', 'prd.stream_temp'], dtype='object')

In [181]:
geodict = geometries.to_geo_dict()

In [179]:
gcm1_flat


,COMID,date,prd.stream_temp
0,23026708,1950-01-01,NaN
1,23026708,1950-01-02,NaN
2,23026708,1950-01-03,1.440884
3,23026708,1950-01-04,0.782451
4,23026708,1950-01-05,0.000000
...,...,...,...
2903706,947010262,2099-12-27,4.693037
2903707,947010262,2099-12-28,5.052310
2903708,947010262,2099-12-29,5.240782
2903709,947010262,2099-12-30,5.344503


In [190]:
geodict.keys()

dict_keys(['type', 'features'])

### Workflow:
1. Create geopackage from nhd, with comid and geometries as columns.
2. Check. (re-open, list layers)
3. Append each gcm as new layer.

In [195]:
flowlines

,COMID,FDATE,RESOLUTION,GNIS_ID,GNIS_NAME,LENGTHKM,REACHCODE,FLOWDIR,WBAREACOMI,FTYPE,FCODE,SHAPE_LENG,ENABLED,GNIS_NBR,geometry
0,9301535,1999-10-08,Medium,None,None,0.999,09040002011583,Uninitialized,0,StreamRiver,46006,0.011661,True,0,"LINESTRING Z (-114.05855 48.99999 0, -114.0583..."
1,22968128,2001-03-02,Medium,None,None,0.134,10030104009812,Uninitialized,0,StreamRiver,46003,0.001671,True,0,"LINESTRING Z (-113.00089 47.46274 0, -113.0020..."
2,22968138,2001-03-02,Medium,None,None,0.217,10030104009813,Uninitialized,0,StreamRiver,46003,0.002145,True,0,"LINESTRING Z (-113.0013 47.45837 0, -113.00137..."
3,23135995,2001-02-07,Medium,None,None,0.916,16010102003533,Uninitialized,0,StreamRiver,46003,0.010671,True,0,"LINESTRING Z (-111.06214 42.50841 0, -111.0619..."
4,23135997,2001-02-07,Medium,None,None,1.865,16010102003534,Uninitialized,0,StreamRiver,46003,0.019193,True,0,"LINESTRING Z (-111.04516 42.51372 0, -111.0447..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272335,24545039,2005-08-27,Medium,None,None,2.847,17120009001542,Uninitialized,0,StreamRiver,46003,0.029718,True,0,"LINESTRING Z (-118.12671 42.98327 0, -118.1260..."
272336,24545041,2005-08-27,Medium,None,None,2.005,17120009001543,Uninitialized,0,StreamRiver,46003,0.020023,True,0,"LINESTRING Z (-118.1111 42.99525 0, -118.11175..."
272337,23706720,2001-02-28,Medium,None,None,4.217,18010201002017,Uninitialized,0,StreamRiver,46003,0.043621,True,0,"LINESTRING Z (-121.60348 43.32724 0, -121.6034..."
272338,23931942,2001-03-14,Medium,1127294,South Fork Keene Creek,0.029,18010206012384,Uninitialized,0,StreamRiver,46006,0.000284,True,0,"LINESTRING Z (-122.47671 42.08355 0, -122.4765..."


In [196]:
gcm1_flat

,COMID,date,prd.stream_temp
0,23026708,1950-01-01,NaN
1,23026708,1950-01-02,NaN
2,23026708,1950-01-03,1.440884
3,23026708,1950-01-04,0.782451
4,23026708,1950-01-05,0.000000
...,...,...,...
2903706,947010262,2099-12-27,4.693037
2903707,947010262,2099-12-28,5.052310
2903708,947010262,2099-12-29,5.240782
2903709,947010262,2099-12-30,5.344503


In [199]:
cur_flowlines = flowlines.loc[flowlines.COMID.isin(cur_comids)]

In [200]:
cur_flowlines


,COMID,FDATE,RESOLUTION,GNIS_ID,GNIS_NAME,LENGTHKM,REACHCODE,FLOWDIR,WBAREACOMI,FTYPE,FCODE,SHAPE_LENG,ENABLED,GNIS_NBR,geometry
39625,23026708,2001-02-27,Medium,1522311,Little Spokane River,6.579,17010308000018,With Digitized,0,StreamRiver,46006,0.074359,True,0,"LINESTRING Z (-117.49237 47.78126 0, -117.4922..."
39626,23026710,2001-02-27,Medium,1522311,Little Spokane River,1.730,17010308000019,With Digitized,0,StreamRiver,46006,0.019356,True,0,"LINESTRING Z (-117.48148 47.77784 0, -117.4820..."
39627,23026712,2001-02-27,Medium,1522311,Little Spokane River,0.552,17010308000020,With Digitized,0,StreamRiver,46006,0.005898,True,0,"LINESTRING Z (-117.47747 47.77405 0, -117.4784..."
39628,23026714,2001-02-27,Medium,1522311,Little Spokane River,4.881,17010308000021,With Digitized,0,StreamRiver,46006,0.056356,True,0,"LINESTRING Z (-117.43736 47.77257 0, -117.4390..."
39629,23026716,2001-02-27,Medium,1522311,Little Spokane River,2.185,17010308000022,With Digitized,0,StreamRiver,46006,0.024544,True,0,"LINESTRING Z (-117.42139 47.78146 0, -117.4217..."
39630,23026718,2001-02-27,Medium,1522311,Little Spokane River,0.542,17010308000023,With Digitized,0,StreamRiver,46006,0.006188,True,0,"LINESTRING Z (-117.41655 47.78288 0, -117.4168..."
39631,23026720,2001-02-27,Medium,1522311,Little Spokane River,3.435,17010308000024,With Digitized,0,StreamRiver,46006,0.040026,True,0,"LINESTRING Z (-117.38456 47.79559 0, -117.3846..."
39632,23026722,2001-02-27,Medium,1504371,Deadman Creek,0.335,17010308000025,With Digitized,0,StreamRiver,46006,0.004426,True,0,"LINESTRING Z (-117.38015 47.79566 0, -117.3819..."
39633,23026724,2001-02-27,Medium,1504371,Deadman Creek,2.626,17010308000026,With Digitized,0,StreamRiver,46006,0.027978,True,0,"LINESTRING Z (-117.36474 47.77768 0, -117.3647..."
39634,23026726,2001-02-27,Medium,1504371,Deadman Creek,2.512,17010308000027,With Digitized,0,StreamRiver,46006,0.030440,True,0,"LINESTRING Z (-117.33766 47.78557 0, -117.3389..."


In [201]:
len(cur_flowlines)

53

In [202]:
cur_flowlines.to_file('outputlines.gpkg', layer = 'flowlines', driver = 'GPKG')

In [205]:
test_lines = gpd.read_file('outputlines.gpkg')

In [38]:
gpd.list_layers('../DATA/Riverscapes/Riverscapes_Context-Riversapes_Context_for_Lower_Umpqua_River/hydrology/hydro_derivatives.gpkg')

,name,geometry_type
0,buffered_clip100m,Polygon
1,buffered_clip500m,Polygon
2,NHDAreaSplit,Polygon Z
3,NHDWaterbodySplit,Polygon Z
4,processing_extent,Polygon
5,catchments,Polygon
6,network_crossings,Point
7,network_intersected,LineString
8,network_segmented,LineString


In [209]:
test_lines.explore()

In [210]:
predsdf

prd.stream_temp
COMID     GCM date                       
23026708  1   1950-01-01              NaN
              1950-01-02              NaN
              1950-01-03         1.440884
              1950-01-04         0.782451
              1950-01-05         0.000000
...                                   ...
947010262 10  2099-12-27         6.673878
              2099-12-28         6.390356
              2099-12-29         5.624401
              2099-12-30         7.018028
              2099-12-31         5.834494

[29037110 rows x 1 columns]

In [211]:
gcm1_flat.to_file('outputlines.gpkg', layer="gcm1", mode="a", driver="GPKG")

AttributeError: 'DataFrame' object has no attribute 'to_file'

In [213]:
import sqlite3

In [214]:
conn = sqlite3.connect('outputlines.gpkg')

In [215]:
gcm1_flat.to_sql('gcm1', conn, index = False)
conn.close()

In [216]:
gpd.list_layers('outputlines.gpkg')

/opt/miniconda3/envs/geopd-env/lib/python3.12/site-packages/pyogrio/core.py:130: RuntimeWarning: Field format 'TIMESTAMP' not supported
  return ogr_list_layers(get_vsi_path_or_buffer(path_or_buffer))


,name,geometry_type
0,flowlines,LineString Z
1,gcm1,None


In [218]:
gpd.read_file('outputlines.gpkg', layer = 'gcm1')

/opt/miniconda3/envs/geopd-env/lib/python3.12/site-packages/pyogrio/raw.py:198: RuntimeWarning: Field format 'TIMESTAMP' not supported
  return ogr_read(


,COMID,date,prd.stream_temp
0,23026708,1950-01-01 00:00:00,NaN
1,23026708,1950-01-02 00:00:00,NaN
2,23026708,1950-01-03 00:00:00,1.440884
3,23026708,1950-01-04 00:00:00,0.782451
4,23026708,1950-01-05 00:00:00,0.000000
...,...,...,...
2903706,947010262,2099-12-27 00:00:00,4.693037
2903707,947010262,2099-12-28 00:00:00,5.052310
2903708,947010262,2099-12-29 00:00:00,5.240782
2903709,947010262,2099-12-30 00:00:00,5.344503
